# Crypto Trading Strategy Monte Carlo Analysis

This notebook performs comprehensive Monte Carlo simulations on crypto trading strategies.

## Key Metrics Calculated:
- **Monte Carlo Simulation**: Multiple backtest runs to assess strategy robustness
- **Sharpe Ratio**: Risk-adjusted return metric (higher is better, >1.0 is good)
- **Expected Value**: Expected profit/loss per trade (positive = profitable strategy)
- **Sortino Ratio**: Downside risk-adjusted return
- **Pass Rate**: Percentage of simulations that pass funded account rules


In [4]:
# Import Daily Bias 4H Strategy Framework
import sys
import os

# Add current directory to path to import the strategy module
if os.path.dirname(os.path.abspath('daily_bias_4h_strategy.py')) not in sys.path:
    sys.path.insert(0, os.path.dirname(os.path.abspath('daily_bias_4h_strategy.py')))

try:
    from daily_bias_4h_strategy import strategy as daily_bias_strategy
    print("✅ Daily Bias 4H Strategy imported successfully!")
    print("   Strategy includes:")
    print("   - 4H Session-based bias detection")
    print("   - CISD (Change in Structure Direction) on 5m timeframe")
    print("   - Multi-timeframe confirmation (4H bias + 5m CISD)")
    print("   - Session reversal patterns (Asia, London, NY 6am/10am)")
except ImportError as e:
    print(f"⚠️ Warning: Could not import daily_bias_4h_strategy: {e}")
    daily_bias_strategy = None


✅ Daily Bias 4H Strategy imported successfully!
   Strategy includes:
   - 4H Session-based bias detection
   - CISD (Change in Structure Direction) on 5m timeframe
   - Multi-timeframe confirmation (4H bias + 5m CISD)
   - Session reversal patterns (Asia, London, NY 6am/10am)


## 2. Backtesting Engine

This class handles backtesting with account rules and risk metrics.


In [5]:
class CryptoBacktester:
    """
    Backtesting engine for crypto trading strategies with funded account rules.
    """
    
    def __init__(self, account_size=5000, daily_dd_limit=0.02, max_dd_limit=0.03,
                 entry_size=4800, target_gain=500):
        self.account_size = account_size
        self.daily_dd_limit = daily_dd_limit
        self.max_dd_limit = max_dd_limit
        self.entry_size = entry_size
        self.target_gain = target_gain
        
    def run_backtest(self, trades_df, strategy_func=None):
        """
        Run backtest on trades DataFrame.
        
        Parameters:
        -----------
        trades_df : pd.DataFrame
            DataFrame with columns: entry_price, exit_price, entry_time, symbol
        strategy_func : function, optional
            Custom strategy function that returns ('BUY', confidence) or ('HOLD', confidence)
            
        Returns:
        --------
        dict : Backtest results with metrics
        """
        equity_curve = [self.account_size]
        trade_log = []
        current_equity = self.account_size
        peak_equity = self.account_size
        daily_peak = self.account_size
        current_date = None
        
        # Pre-filter trades if strategy function exists (more efficient than checking in loop)
        if strategy_func:
            # Apply strategy to all trades upfront (vectorized approach)
            valid_trades = []
            for idx, trade in trades_df.iterrows():
                try:
                    signal, confidence = strategy_func(trade, {'equity': self.account_size})
                    if signal == 'BUY':
                        valid_trades.append(idx)
                except:
                    continue
            if len(valid_trades) == 0:
                # No valid trades, return early
                return {
                    'final_equity': self.account_size,
                    'total_pnl': 0,
                    'return_pct': 0,
                    'num_trades': 0,
                    'max_dd_pct': 0,
                    'sharpe': 0,
                    'sortino': 0,
                    'expected_value': 0,
                    'win_rate': 0,
                    'winning_trades': 0,
                    'losing_trades': 0,
                    'equity_curve': [self.account_size],
                    'trade_log': [],
                    'passed': False
                }
            trades_df = trades_df.loc[valid_trades].copy()
        else:
            # No strategy function - filter by signal column if it exists
            if 'signal' in trades_df.columns:
                trades_df = trades_df[trades_df['signal'].isin(['BUY', 'SELL'])].copy()
        
        # Convert to numpy arrays for faster processing
        entry_prices = trades_df['entry_price'].values
        exit_prices = trades_df['exit_price'].values
        sides = trades_df.get('side', 'LONG').values if 'side' in trades_df.columns else np.full(len(trades_df), 'LONG')
        
        # Process trades in vectorized batches where possible
        for i in range(len(trades_df)):
            entry_price = entry_prices[i]
            exit_price = exit_prices[i]
            side = sides[i] if isinstance(sides, np.ndarray) else 'LONG'
            
            # Calculate PnL (handle both LONG and SHORT)
            if side == 'SHORT':
                # For shorts, profit when price goes down
                pnl = self.entry_size * ((entry_price - exit_price) / entry_price)
            else:
                # For longs, profit when price goes up
                pnl = self.entry_size * ((exit_price - entry_price) / entry_price)
            
            new_equity = current_equity + pnl
            
            # Check daily DD
            if daily_peak > 0:
                daily_dd = (daily_peak - new_equity) / daily_peak
                if daily_dd > self.daily_dd_limit:
                    break
            
            # Check max DD
            if peak_equity > 0:
                max_dd = (peak_equity - new_equity) / peak_equity
                if max_dd > self.max_dd_limit:
                    break
            
            current_equity = new_equity
            if current_equity > peak_equity:
                peak_equity = current_equity
            if current_equity > daily_peak:
                daily_peak = current_equity
            
            trade_log.append({
                'entry_price': entry_price,
                'exit_price': exit_price,
                'pnl': pnl,
                'equity': current_equity
            })
            
            equity_curve.append(current_equity)
            
            # Check target gain
            if current_equity - self.account_size >= self.target_gain:
                break
        
        # Calculate metrics
        equity_array = np.array(equity_curve)
        total_pnl = current_equity - self.account_size
        return_pct = (total_pnl / self.account_size) * 100
        
        # Drawdown calculation
        rolling_max = np.maximum.accumulate(equity_array)
        drawdowns = (rolling_max - equity_array) / rolling_max
        max_dd = drawdowns.max() * 100 if len(drawdowns) > 0 else 0
        
        # Returns for Sharpe/Sortino
        returns = np.diff(equity_array) / equity_array[:-1]
        returns = returns[~np.isnan(returns)]
        
        # Sharpe Ratio (annualized, assuming 312 trading days)
        if len(returns) > 1 and returns.std() > 0:
            sharpe = (returns.mean() * np.sqrt(312)) / returns.std()
        else:
            sharpe = 0
        
        # Sortino Ratio (only downside volatility)
        downside_returns = returns[returns < 0]
        if len(downside_returns) > 1 and downside_returns.std() > 0:
            sortino = (returns.mean() * np.sqrt(312)) / downside_returns.std()
        else:
            sortino = 0
        
        # Expected Value (average PnL per trade)
        if len(trade_log) > 0:
            expected_value = np.mean([t['pnl'] for t in trade_log])
        else:
            expected_value = 0
        
        # Calculate Win Rate
        if len(trade_log) > 0:
            winning_trades = sum(1 for t in trade_log if t['pnl'] > 0)
            losing_trades = sum(1 for t in trade_log if t['pnl'] < 0)
            total_trades = len(trade_log)
            win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0
        else:
            winning_trades = 0
            losing_trades = 0
            win_rate = 0
        
        # Determine if passed (must meet BOTH criteria)
        passed = total_pnl >= self.target_gain and max_dd < self.max_dd_limit * 100
        
        return {
            'final_equity': current_equity,
            'total_pnl': total_pnl,
            'return_pct': return_pct,
            'num_trades': len(trade_log),
            'max_dd_pct': max_dd,
            'sharpe': sharpe,
            'sortino': sortino,
            'expected_value': expected_value,
            'win_rate': win_rate,
            'winning_trades': winning_trades,
            'losing_trades': losing_trades,
            'equity_curve': equity_curve,
            'trade_log': trade_log,
            'passed': passed,
            'failed_reason': 'target_not_met' if total_pnl < self.target_gain else ('dd_exceeded' if max_dd >= self.max_dd_limit * 100 else 'passed')
        }

print("Backtesting engine defined!")


Backtesting engine defined!


## 3. Prepare Trade Data

Convert price data into trade format for backtesting.


In [6]:
# Override: Use ALL available data points (if not already done)
if len(trades_df) > 0 and len(trades_df) < 10000:
    # Re-prepare with all data if we have less than 10k trades
    if 'price' in locals() and len(price) > 0 and 'Ticker' in price.columns:
        btc_data = price[price['Ticker'] == 'BTC'].copy()
        trades_df = prepare_trades_from_data(btc_data, symbol='BTC', sample_interval=1, max_trades=999999)
        print(f"✅ Updated: Using ALL {len(trades_df):,} available trades from CSV data")
        print(f"   Date range: {trades_df['entry_time'].min()} to {trades_df['entry_time'].max()}")


NameError: name 'trades_df' is not defined

In [ ]:
# Prepare trade data from price data
CENTRAL_TZ = pytz.timezone('America/Chicago')  # Central Timezone (CST/CDT)

def prepare_trades_from_data(price_df, symbol='BTC', sample_interval=5, max_trades=1000):
    """
    Convert price data to trade format.
    """
    # Filter to trading hours
    price_df['central_time'] = price_df.index.tz_localize('UTC').tz_convert(CENTRAL_TZ)
    price_df['hour'] = price_df['central_time'].dt.hour
    price_df['minute'] = price_df['central_time'].dt.minute
    price_df['weekday'] = price_df['central_time'].dt.weekday  # Monday=0, Sunday=6
    
    # Filter to 7:00 AM - 11:30 AM Central timezone, Monday-Saturday
    weekday_mask = price_df['weekday'] < 6
    # Filter to 7:00 AM - 11:30 AM Central timezone
    time_mask = ((price_df['hour'] == 7) & (price_df['minute'] >= 0)) | \
                ((price_df['hour'] >= 8) & (price_df['hour'] < 11)) | \
                ((price_df['hour'] == 11) & (price_df['minute'] <= 30))
    mask = weekday_mask & time_mask
    
    window_data = price_df[mask].copy()
    
    if len(window_data) == 0:
        return pd.DataFrame()
    
    # Sample every Nth bar
    window_data = window_data.iloc[::sample_interval].head(max_trades)
    
    # Create trades DataFrame
    trades = []
    for i in range(1, len(window_data)):
        current = window_data.iloc[i]
        previous = window_data.iloc[i-1]
        
        trades.append({
            'symbol': symbol,
            'entry_price': previous['Close'],
            'exit_price': current['Close'],
            'entry_time': previous.name,
            'high': current['High'],
            'low': current['Low'],
            'volume': current['Volume'],
            'price_range': current['High'] - current['Low'],
            'cvd': 0,  # Simplified
            'cvd_ma': 0,
            'recent_high': window_data['High'].iloc[max(0, i-20):i].max(),
            'recent_low': window_data['Low'].iloc[max(0, i-20):i].min(),
        })
    
    return pd.DataFrame(trades)

# Prepare trades for BTC
if len(price) > 0 and 'Ticker' in price.columns:
    btc_data = price[price['Ticker'] == 'BTC'].copy()
    trades_df = prepare_trades_from_data(btc_data, symbol='BTC', sample_interval=5, max_trades=500)
    print(f"Prepared {len(trades_df)} trades from BTC data")
    print(f"Date range: {trades_df['entry_time'].min()} to {trades_df['entry_time'].max()}")
else:
    print("No price data available. Please load data first.")
    trades_df = pd.DataFrame()


## 3.5 CISD Multi-Timeframe Strategy Implementation

This section implements the CISD (Cumulative Imbalance Supply/Demand) strategy with multi-timeframe analysis:
- **4H Chart**: Session-profile model to define bias (6am-10am NYC session)
- **5m Chart**: CISD signals within the active 4H candle
- **Entry Rules**: CISD signals aligned with 4H bias


In [ ]:
# ============================================================================
# PREPARE MULTI-TIMEFRAME DATA FOR CISD STRATEGY
# ============================================================================

def prepare_multitimeframe_data(price_df, symbol='BTC'):
    """
    Prepare 4H and 5m timeframe data for CISD strategy.
    
    Parameters:
    -----------
    price_df : pd.DataFrame
        Minute-by-minute price data
    symbol : str
        Trading symbol
        
    Returns:
    --------
    tuple: (htf_data_4h, ltf_data_5m)
    """
    if len(price_df) == 0:
        return pd.DataFrame(), pd.DataFrame()
    
    # Filter to symbol if Ticker column exists
    if 'Ticker' in price_df.columns:
        symbol_data = price_df[price_df['Ticker'] == symbol].copy()
    else:
        symbol_data = price_df.copy()
    
    if len(symbol_data) == 0:
        return pd.DataFrame(), pd.DataFrame()
    
    # Ensure timezone-aware index
    if symbol_data.index.tzinfo is None:
        symbol_data.index = symbol_data.index.tz_localize('UTC')
    
    # Resample to 4H timeframe (higher timeframe)
    htf_4h = symbol_data.resample('4H', label='right', closed='right').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    }).dropna()
    
    # Resample to 5m timeframe (lower timeframe)
    ltf_5m = symbol_data.resample('5min', label='right', closed='right').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    }).dropna()
    
    # Filter 5m data to entry window (6am-10am NYC)
    ltf_5m_nyc = ltf_5m.copy()
    ltf_5m_nyc.index = ltf_5m_nyc.index.tz_convert(NYC_TZ)
    ltf_5m_nyc['hour'] = ltf_5m_nyc.index.hour
    ltf_5m_nyc = ltf_5m_nyc[(ltf_5m_nyc['hour'] >= 6) & (ltf_5m_nyc['hour'] < 10)]
    ltf_5m_nyc = ltf_5m_nyc.drop('hour', axis=1)
    ltf_5m_nyc.index = ltf_5m_nyc.index.tz_convert('UTC')
    
    return htf_4h, ltf_5m_nyc

# Prepare multi-timeframe data
if len(price) > 0:
    htf_4h_data, ltf_5m_data = prepare_multitimeframe_data(price, symbol='BTC')
    print(f"✅ Prepared multi-timeframe data:")
    print(f"   4H data: {len(htf_4h_data)} candles")
    print(f"   5m data (6am-10am NYC): {len(ltf_5m_data)} candles")
    print(f"   4H date range: {htf_4h_data.index.min()} to {htf_4h_data.index.max()}")
    print(f"   5m date range: {ltf_5m_data.index.min()} to {ltf_5m_data.index.max()}")
else:
    htf_4h_data = pd.DataFrame()
    ltf_5m_data = pd.DataFrame()
    print("⚠️ No price data available for multi-timeframe preparation")


In [ ]:
# ============================================================================
# CONVERT 5M DATA TO TRADES USING CISD STRATEGY
# ============================================================================

def prepare_cisd_trades(ltf_5m_data, htf_4h_data, symbol='BTC', max_trades=1000):
    """
    Convert 5m data to trades using CISD strategy signals.
    
    Parameters:
    -----------
    ltf_5m_data : pd.DataFrame
        5-minute timeframe data (filtered to entry window)
    htf_4h_data : pd.DataFrame
        4-hour timeframe data for bias determination
    symbol : str
        Trading symbol
    max_trades : int
        Maximum number of trades to generate
        
    Returns:
    --------
    pd.DataFrame : Trades DataFrame
    """
    if len(ltf_5m_data) == 0 or len(htf_4h_data) == 0:
        return pd.DataFrame()
    
    trades = []
    lookback_window = 20  # Bars to look back for CISD detection
    
    for i in range(lookback_window + 1, len(ltf_5m_data)):
        if len(trades) >= max_trades:
            break
        
        current_bar = ltf_5m_data.iloc[i]
        previous_bars = ltf_5m_data.iloc[max(0, i-lookback_window):i]
        current_time = ltf_5m_data.index[i]
        
        # Check for entry signal using CISD strategy
        signal, confidence = cisd_strategy.check_entry_signal(
            current_bar, previous_bars, current_time, htf_4h_data
        )
        
        # Only create trade if signal is BUY or SELL
        if signal in ['BUY', 'SELL']:
            # For exit, use next bar's close (or a few bars ahead)
            exit_idx = min(i + 5, len(ltf_5m_data) - 1)  # Exit 5 bars later (25 minutes)
            exit_bar = ltf_5m_data.iloc[exit_idx]
            
            entry_price = current_bar['Close']
            exit_price = exit_bar['Close']
            
            # Adjust for SELL signals (short)
            if signal == 'SELL':
                # For shorts, profit when price goes down
                pnl_multiplier = -1
            else:
                pnl_multiplier = 1
            
            trades.append({
                'symbol': symbol,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'entry_time': current_time,
                'exit_time': ltf_5m_data.index[exit_idx],
                'high': current_bar['High'],
                'low': current_bar['Low'],
                'volume': current_bar['Volume'],
                'price_range': current_bar['High'] - current_bar['Low'],
                'signal': signal,
                'confidence': confidence,
                'side': 'LONG' if signal == 'BUY' else 'SHORT',
                'recent_high': previous_bars['High'].max(),
                'recent_low': previous_bars['Low'].min(),
            })
    
    return pd.DataFrame(trades)

# Prepare CISD trades
if len(ltf_5m_data) > 0 and len(htf_4h_data) > 0:
    cisd_trades_df = prepare_cisd_trades(ltf_5m_data, htf_4h_data, symbol='BTC', max_trades=500)
    print(f"\n✅ Generated {len(cisd_trades_df)} CISD strategy trades")
    if len(cisd_trades_df) > 0:
        print(f"   Date range: {cisd_trades_df['entry_time'].min()} to {cisd_trades_df['entry_time'].max()}")
        print(f"   BUY signals: {len(cisd_trades_df[cisd_trades_df['signal'] == 'BUY'])}")
        print(f"   SELL signals: {len(cisd_trades_df[cisd_trades_df['signal'] == 'SELL'])}")
        print(f"\n   First few trades:")
        print(cisd_trades_df[['entry_time', 'entry_price', 'exit_price', 'signal', 'confidence']].head())
else:
    cisd_trades_df = pd.DataFrame()
    print("⚠️ No multi-timeframe data available for CISD trade generation")


In [ ]:
# ============================================================================
# STRATEGY SELECTION - Daily Bias 4H Framework
# ============================================================================

# Strategy selection: Use Daily Bias 4H Strategy Framework
USE_DAILY_BIAS_STRATEGY = True  # Set to True to use Daily Bias Framework

if USE_DAILY_BIAS_STRATEGY and daily_bias_strategy is not None:
    print("✅ Using Daily Bias 4H Strategy Framework")
    print("   Strategy features:")
    print("   - 4H Session-based bias detection (Asia, London, NY sessions)")
    print("   - CISD (Change in Structure Direction) on 5m timeframe")
    print("   - Multi-timeframe confirmation (4H bias + 5m CISD signals)")
    print("   - Session reversal patterns (Asia Reversal, London Reversal, NY 6am/10am)")
    print("   - Trading window: 7:00 AM - 11:30 AM Central timezone, Mon-Sat")
    
    # Use Daily Bias strategy function
    strategy_func = daily_bias_strategy
    
    # Prepare trades from data if not already prepared
    if 'trades_df' not in locals() or len(trades_df) == 0:
        print("   Preparing trades from historical data...")
        trades_df = prepare_trades_from_data(price, symbol='BTC', max_trades=1000)
        print(f"   Prepared {len(trades_df)} trades")
else:
    print("⚠️ Daily Bias Strategy not available, using fallback strategy")
    if 'cisd_trades_df' in locals() and len(cisd_trades_df) > 0:
        print("   Falling back to CISD Multi-Timeframe Strategy")
        trades_df = cisd_trades_df.copy()
        strategy_func = None
    else:
        print("   Using simple momentum strategy")
        strategy_func = my_strategy if 'my_strategy' in globals() else None


## 4. Define Trading Strategy

Choose between:
- **CISD Multi-Timeframe Strategy**: Uses 4H bias + 5m CISD signals (recommended)
- **Simple Momentum Strategy**: Basic momentum-based entry


## 4. Define Trading Strategy

Define your custom trading strategy here. The function should return ('BUY', confidence) or ('HOLD', confidence).


In [ ]:
# ============================================================================
# STRATEGY PARAMETERS - MODIFY THESE TO CHANGE STRATEGY BEHAVIOR
# ============================================================================

STRATEGY_PARAMS = {
    # Thresholds for position in price range (0.0 to 1.0)
    'high_threshold': 0.8,          # Near recent high - strong momentum
    'mid_threshold': 0.6,           # Upper portion of range - moderate momentum
    
    # Confidence levels (0.0 to 1.0)
    'high_confidence': 0.85,         # Confidence for high threshold signals
    'mid_confidence': 0.70,          # Confidence for mid threshold signals
    'low_confidence': 0.50,         # Confidence for hold signals
    
    # Volume filter (optional)
    'min_volume_factor': None,       # Set to a number to enable volume filtering
}

print("STRATEGY PARAMETERS LOADED")
print("=" * 70)
for key, value in STRATEGY_PARAMS.items():
    if isinstance(value, float):
        print(f"  {key:25s}: {value:.4f}")
    else:
        print(f"  {key:25s}: {value}")
print("=" * 70)


In [ ]:
# ============================================================================
# CONFIGURABLE STRATEGY FUNCTION - Uses STRATEGY_PARAMS
# ============================================================================

def my_strategy(row, context=None):
    """
    Configurable trading strategy using STRATEGY_PARAMS.
    
    Modify STRATEGY_PARAMS above to change strategy behavior.
    
    Parameters:
    -----------
    row : pd.Series
        Trade data with entry_price, exit_price, recent_high, recent_low, volume, etc.
    context : dict, optional
        Additional context like current equity
        
    Returns:
    --------
    tuple: (signal, confidence)
        signal: 'BUY' or 'HOLD'
        confidence: float between 0 and 1
    """
    entry_price = row['entry_price']
    recent_high = row.get('recent_high', entry_price * 1.02)
    recent_low = row.get('recent_low', entry_price * 0.98)
    volume = row.get('volume', 0)
    
    # Calculate price range
    price_range = recent_high - recent_low
    if price_range == 0:
        price_range = entry_price * 0.01  # Default 1% range
    
    # Calculate position in range (0.0 = at low, 1.0 = at high)
    position_in_range = (entry_price - recent_low) / price_range
    
    # Apply volume filter if enabled
    if STRATEGY_PARAMS['min_volume_factor'] is not None:
        # Simple volume check (you can enhance this)
        if volume < 100:  # Basic threshold
            return 'HOLD', STRATEGY_PARAMS['low_confidence']
    
    # Strategy logic based on position in price range
    if position_in_range >= STRATEGY_PARAMS['high_threshold']:
        # Near recent high - strong momentum
        return 'BUY', STRATEGY_PARAMS['high_confidence']
    
    elif position_in_range >= STRATEGY_PARAMS['mid_threshold']:
        # In upper portion of range - moderate momentum
        return 'BUY', STRATEGY_PARAMS['mid_confidence']
    
    else:
        # Near recent low - weak momentum, hold
        return 'HOLD', STRATEGY_PARAMS['low_confidence']

print("✅ Strategy function defined using STRATEGY_PARAMS")
print("   Re-run this cell after modifying STRATEGY_PARAMS to update the strategy!")


## 4.2 Backtesting Parameters Configuration

**Modify these to test different account rules and risk settings:**

In [ ]:
# ============================================================================
# BACKTESTING PARAMETERS - MODIFY THESE TO TEST DIFFERENT ACCOUNT RULES
# ============================================================================

BACKTEST_PARAMS = {
    # Account settings
    'account_size': 5000,          # Starting account size ($)
    'entry_size': 4800,             # Position size per trade ($)
    'target_gain': 500,             # Target profit to reach ($)
    
    # Risk limits (as decimals: 0.02 = 2%)
    'daily_dd_limit': 0.02,         # Maximum daily drawdown (2%)
    'max_dd_limit': 0.03,           # Maximum total drawdown (3%)
    
    # Monte Carlo settings
    'num_simulations': 500,         # Number of Monte Carlo runs
    'sample_size': 100,             # Number of trades per simulation
}

print("=" * 70)
print("BACKTESTING PARAMETERS LOADED")
print("=" * 70)
for key, value in BACKTEST_PARAMS.items():
    if isinstance(value, float):
        print(f"  {key:25s}: {value:.4f} ({value*100:.2f}%)")
    else:
        print(f"  {key:25s}: {value}")
print("=" * 70)


## 4.3 Interactive Parameter Testing

**Modify the parameters below and run this cell to see how they affect your strategy performance.**

This section allows you to test different parameter combinations and visualize their impact on:
- Expected Value (profitability)
- Pass Rate (funded account success rate)
- Sharpe Ratio (risk-adjusted returns)
- Drawdown metrics

Adjust the values in the code cell below and re-run to see updated results.

## ⚠️ IMPORTANT FIX FOR KeyError

**If you get a KeyError for 'sharpe_ratio', 'max_drawdown', or 'final_equity':**

The backtester returns columns with different names. Run the fix cell below BEFORE running the "Quick test run" cell, OR modify the "Quick test run" cell to add these lines right after `test_df = pd.DataFrame(test_results)`:

```python
# Add these lines right after test_df = pd.DataFrame(test_results)
test_df['final_equity'] = test_df['equity_curve'].apply(lambda x: x[-1] if len(x) > 0 else TEST_PARAMS['account_size'])
test_df['sharpe_ratio'] = test_df['sharpe']
test_df['sortino_ratio'] = test_df['sortino']
test_df['max_drawdown'] = test_df['max_dd_pct'] / 100
```


In [ ]:
# IMPORTANT: Run this cell BEFORE running the "Quick test run" cell
# This ensures column mappings are available when test_df is created

def add_backtester_columns(df, account_size=5000):
    """Add calculated columns immediately after DataFrame creation."""
    if df is None or len(df) == 0:
        return df
    # Add calculated columns for compatibility
    if 'equity_curve' in df.columns:
        df['final_equity'] = df['equity_curve'].apply(lambda x: x[-1] if len(x) > 0 else account_size)
    if 'sharpe' in df.columns:
        df['sharpe_ratio'] = df['sharpe']
    if 'sortino' in df.columns:
        df['sortino_ratio'] = df['sortino']
    if 'max_dd_pct' in df.columns:
        df['max_drawdown'] = df['max_dd_pct'] / 100
    return df

print("✅ Helper function 'add_backtester_columns' is ready to use")


In [ ]:
# Fix column names in test_df and test_results_df if they exist
# This fixes the KeyError by mapping backtester column names to expected names

def fix_backtester_columns(df, account_size=5000):
    """Add calculated columns for compatibility with backtester results."""
    if df is None or len(df) == 0:
        return df
    
    # Add calculated columns for compatibility (backtester returns different key names)
    if 'final_equity' not in df.columns and 'equity_curve' in df.columns:
        df['final_equity'] = df['equity_curve'].apply(lambda x: x[-1] if len(x) > 0 else account_size)
    if 'sharpe_ratio' not in df.columns and 'sharpe' in df.columns:
        df['sharpe_ratio'] = df['sharpe']
    if 'sortino_ratio' not in df.columns and 'sortino' in df.columns:
        df['sortino_ratio'] = df['sortino']
    if 'max_drawdown' not in df.columns and 'max_dd_pct' in df.columns:
        df['max_drawdown'] = df['max_dd_pct'] / 100  # Convert percentage to decimal
    return df

# Fix test_df if it exists (created in the quick test cell)
if 'test_df' in globals() and len(test_df) > 0:
    account_size = TEST_PARAMS.get('account_size', 5000) if 'TEST_PARAMS' in globals() else 5000
    test_df = fix_backtester_columns(test_df, account_size)
    print("✅ Fixed column names in test_df")

# Fix test_results_df if it exists
if 'test_results_df' in globals() and len(test_results_df) > 0:
    account_size = TEST_PARAMS.get('account_size', 5000) if 'TEST_PARAMS' in globals() else 5000
    test_results_df = fix_backtester_columns(test_results_df, account_size)
    print("✅ Fixed column names in test_results_df")


## 5. Run Monte Carlo Simulation

Run the simulation with your current parameter settings.


In [ ]:
# ============================================================================
# RUN MONTE CARLO SIMULATION WITH CURRENT PARAMETERS
# ============================================================================

# Initialize backtester with current parameters
backtester = CryptoBacktester(
    account_size=BACKTEST_PARAMS['account_size'],
    daily_dd_limit=BACKTEST_PARAMS['daily_dd_limit'],
    max_dd_limit=BACKTEST_PARAMS['max_dd_limit'],
    entry_size=BACKTEST_PARAMS['entry_size'],
    target_gain=BACKTEST_PARAMS['target_gain']
)

# Run Monte Carlo simulation
print("=" * 70)
print(f"RUNNING {BACKTEST_PARAMS['num_simulations']} MONTE CARLO SIMULATIONS")
print("=" * 70)
print(f"Strategy Parameters:")
print(f"  High Threshold: {STRATEGY_PARAMS['high_threshold']}")
print(f"  Mid Threshold: {STRATEGY_PARAMS['mid_threshold']}")
print(f"  High Confidence: {STRATEGY_PARAMS['high_confidence']}")
print(f"  Mid Confidence: {STRATEGY_PARAMS['mid_confidence']}")
print(f"\nBacktest Parameters:")
print(f"  Account Size: ${BACKTEST_PARAMS['account_size']:,}")
print(f"  Target Gain: ${BACKTEST_PARAMS['target_gain']:,}")
print(f"  Daily DD Limit: {BACKTEST_PARAMS['daily_dd_limit']*100:.1f}%")
print(f"  Max DD Limit: {BACKTEST_PARAMS['max_dd_limit']*100:.1f}%")
print("=" * 70)
print()

results = []

if len(trades_df) > 0:
    for i in range(BACKTEST_PARAMS['num_simulations']):
        # Randomly sample trades
        sample_trades = trades_df.sample(
            n=min(BACKTEST_PARAMS['sample_size'], len(trades_df)), 
            replace=True
        )
        sample_trades = sample_trades.sort_values('entry_time').reset_index(drop=True)
        
        # Run backtest with strategy
        result = backtester.run_backtest(sample_trades, strategy_func=my_strategy)
        results.append(result)
        
        if (i + 1) % 50 == 0:
            print(f"  Completed {i + 1}/{BACKTEST_PARAMS['num_simulations']} simulations...")
    
    print(f"\n{'='*70}")
    print(f"✓ COMPLETED {BACKTEST_PARAMS['num_simulations']} SIMULATIONS!")
    print(f"{'='*70}")
else:
    print("❌ No trade data available. Please prepare trades first.")
    results = []


## 6. Monte Carlo Results Visualization

Visualize how your strategy parameters affect the simulation results.
hows

In [ ]:
# Example Strategy: Simple momentum-based strategy
def my_strategy(row, context=None):
    """
    Custom trading strategy.
    
    Parameters:
    -----------
    row : pd.Series
        Trade data with entry_price, exit_price, recent_high, recent_low, etc.
    context : dict, optional
        Additional context like current equity
        
    Returns:
    --------
    tuple: (signal, confidence)
        signal: 'BUY' or 'HOLD'
        confidence: float between 0 and 1
    """
    entry_price = row['entry_price']
    recent_high = row.get('recent_high', entry_price * 1.02)
    recent_low = row.get('recent_low', entry_price * 0.98)
    
    # Simple momentum: Buy when price is near recent high
    if entry_price >= recent_high * 0.99:
        return 'BUY', 0.8
    elif entry_price >= recent_low + (recent_high - recent_low) * 0.7:
        return 'BUY', 0.6
    else:
        return 'HOLD', 0.5

print("Strategy defined! Modify 'my_strategy' function to implement your own strategy.")


In [ ]:
# Override: Use maximum data points for Monte Carlo
if len(trades_df) > 0:
    # Update SAMPLE_SIZE to use all available data
    SAMPLE_SIZE = min(len(trades_df), 10000)  # Use up to 10k trades per simulation
    print(f"\n📊 Updated Monte Carlo Configuration:")
    print(f"   Total available trades: {len(trades_df):,}")
    print(f"   Sample size per simulation: {SAMPLE_SIZE:,}")
    print(f"   Number of simulations: {NUM_SIMULATIONS:,}")
    print(f"   Total data points analyzed: {NUM_SIMULATIONS * SAMPLE_SIZE:,}")


## 5. Monte Carlo Simulation

Run multiple backtests with random sampling to assess strategy robustness.


In [ ]:
# ============================================================================
# MONTE CARLO SIMULATION FOR EACH CRYPTOCURRENCY
# ============================================================================
# Run separate Monte Carlo simulations for each crypto (BTC, ETH, SOL, etc.)

NUM_SIMULATIONS = 5000  # Increased to 5000 for more comprehensive analysis
SAMPLE_SIZE = 100  # Number of trades per simulation

# Initialize backtester
# NOTE: These are funded account challenge parameters (very strict)
# For more realistic results, consider:
# - target_gain: 250-300 (5-6% instead of 10%)
# - max_dd_limit: 0.05-0.06 (5-6% instead of 3%)
# - daily_dd_limit: 0.03-0.04 (3-4% instead of 2%)
# - entry_size: 4000-4500 (80-90% instead of 96%)

backtester = CryptoBacktester(
    account_size=5000,
    daily_dd_limit=0.02,      # 2% daily drawdown limit (very strict)
    max_dd_limit=0.03,        # 3% max drawdown limit (very strict)
    entry_size=4800,          # 96% of account per trade (very high)
    target_gain=500            # $500 target = 10% gain (challenging)
)

print("⚠️  NOTE: Using strict funded account challenge parameters")
print(f"   Target Gain: ${backtester.target_gain} ({backtester.target_gain/backtester.account_size*100:.1f}% of account)")
print(f"   Max Drawdown Limit: {backtester.max_dd_limit*100:.1f}% (very strict)")
print(f"   Daily Drawdown Limit: {backtester.daily_dd_limit*100:.1f}% (very strict)")
print(f"   Entry Size: ${backtester.entry_size} ({backtester.entry_size/backtester.account_size*100:.1f}% of account)")
print(f"   Pass Criteria: Must make ${backtester.target_gain} profit AND stay under {backtester.max_dd_limit*100:.1f}% drawdown")
print()

# Check if we have price data with multiple tickers
if 'price' in locals() and len(price) > 0 and 'Ticker' in price.columns:
    tickers = sorted(price['Ticker'].unique())
    print(f"📊 Found {len(tickers)} cryptocurrencies: {', '.join(tickers)}")
    print(f"\n{'='*70}")
    print(f"RUNNING MONTE CARLO SIMULATIONS FOR EACH CRYPTOCURRENCY")
    print(f"{'='*70}\n")
    
    # Store results for each crypto
    crypto_results = {}
    
    for ticker in tickers:
        print(f"\n{'─'*70}")
        print(f"🪙 {ticker} - Preparing trades and running {NUM_SIMULATIONS} simulations...")
        print(f"{'─'*70}")
        
        # Filter data for this ticker
        ticker_data = price[price['Ticker'] == ticker].copy()
        
        # Prepare trades for this ticker (optimize: use sample_interval to reduce data size)
        # Use sample_interval=5 to reduce from ~590k to ~118k trades (5x faster)
        ticker_trades_df = prepare_trades_from_data(
            ticker_data, 
            symbol=ticker, 
            sample_interval=5,  # Sample every 5th bar instead of every bar (5x speedup)
            max_trades=999999
        )
        
        if len(ticker_trades_df) == 0:
            print(f"⚠️ No trades prepared for {ticker}. Skipping...")
            continue
        
        print(f"   Prepared {len(ticker_trades_df):,} trades for {ticker}")
        print(f"   Date range: {ticker_trades_df['entry_time'].min()} to {ticker_trades_df['entry_time'].max()}")
        
        # Determine which trades to use
        if USE_CISD_STRATEGY and 'cisd_trades_df' in globals() and len(cisd_trades_df) > 0:
            # Filter CISD trades for this ticker if available
            ticker_cisd = cisd_trades_df[cisd_trades_df.get('symbol', cisd_trades_df.get('Ticker', '')) == ticker].copy()
            if len(ticker_cisd) > 0:
                print(f"   Using CISD Multi-Timeframe Strategy ({len(ticker_cisd)} trades)")
                mc_trades_df = ticker_cisd
                mc_strategy_func = None
            else:
                print(f"   Using standard trades with strategy")
                mc_trades_df = ticker_trades_df
                mc_strategy_func = daily_bias_strategy if 'daily_bias_strategy' in globals() and daily_bias_strategy else (my_strategy if 'my_strategy' in globals() else None)
        else:
            print(f"   Using standard trades with strategy")
            mc_trades_df = ticker_trades_df
            mc_strategy_func = daily_bias_strategy if 'daily_bias_strategy' in globals() and daily_bias_strategy else (my_strategy if 'my_strategy' in globals() else None)
        
        # Run Monte Carlo simulation for this ticker
        ticker_results = []
        
        # Pre-sample indices for faster processing (avoid repeated sampling overhead)
        import time
        start_time = time.time()
        
        for i in range(NUM_SIMULATIONS):
            # Randomly sample trades (use indices for faster sampling)
            sample_size = min(SAMPLE_SIZE, len(mc_trades_df))
            sample_indices = np.random.choice(len(mc_trades_df), size=sample_size, replace=True)
            sample_trades = mc_trades_df.iloc[sample_indices].copy()
            sample_trades = sample_trades.sort_values('entry_time').reset_index(drop=True)
            
            # Run backtest
            result = backtester.run_backtest(sample_trades, strategy_func=mc_strategy_func)
            result['ticker'] = ticker  # Tag result with ticker
            ticker_results.append(result)
            
            # Progress update with time estimate
            if (i + 1) % 500 == 0:
                elapsed = time.time() - start_time
                rate = (i + 1) / elapsed if elapsed > 0 else 0
                remaining = (NUM_SIMULATIONS - i - 1) / rate if rate > 0 else 0
                print(f"   Completed {i + 1}/{NUM_SIMULATIONS} simulations... "
                      f"({elapsed:.1f}s elapsed, ~{remaining:.1f}s remaining)")
        
        crypto_results[ticker] = ticker_results
        print(f"\n✓ Completed {NUM_SIMULATIONS} simulations for {ticker}!")
    
    # Combine all results for backward compatibility
    results = []
    for ticker_results in crypto_results.values():
        results.extend(ticker_results)
    
    print(f"\n{'='*70}")
    print(f"✅ COMPLETED MONTE CARLO SIMULATIONS FOR ALL CRYPTOCURRENCIES")
    print(f"{'='*70}")
    print(f"Total simulations: {len(results):,}")
    for ticker, ticker_results in crypto_results.items():
        print(f"  {ticker}: {len(ticker_results):,} simulations")
    
else:
    # Fallback to original single simulation if no ticker data
    print("⚠️ No ticker data found. Running single simulation...")
    
    # Determine which trades to use
    if USE_CISD_STRATEGY and len(cisd_trades_df) > 0:
        print(f"📊 Using CISD Multi-Timeframe Strategy")
        print(f"   Available CISD trades: {len(cisd_trades_df)}")
        mc_trades_df = cisd_trades_df.copy()
        mc_strategy_func = None
    else:
        print(f"📊 Using standard trades")
        mc_trades_df = trades_df
        mc_strategy_func = my_strategy if 'my_strategy' in globals() else None
    
    # Run Monte Carlo simulation
    print(f"\nRunning {NUM_SIMULATIONS} Monte Carlo simulations...")
    results = []
    
    if len(mc_trades_df) > 0:
        for i in range(NUM_SIMULATIONS):
            sample_trades = mc_trades_df.sample(n=min(SAMPLE_SIZE, len(mc_trades_df)), replace=True)
            sample_trades = sample_trades.sort_values('entry_time').reset_index(drop=True)
            result = backtester.run_backtest(sample_trades, strategy_func=mc_strategy_func)
            results.append(result)
            
            if (i + 1) % 50 == 0:
                print(f"Completed {i + 1}/{NUM_SIMULATIONS} simulations...")
        
        print(f"\n✓ Completed {NUM_SIMULATIONS} simulations!")
    else:
        print("No trade data available. Please prepare trades first.")


# Plot each cryptocurrency in separate charts
if 'Ticker' in price.columns and len(price) > 0:
    tickers = sorted(price['Ticker'].unique())
    num_tickers = len(tickers)
    
    # Create subplots - one for each cryptocurrency
    fig, axes = plt.subplots(num_tickers, 1, figsize=(14, 5 * num_tickers))
    
    # If only one ticker, axes won't be an array
    if num_tickers == 1:
        axes = [axes]
    
    for idx, ticker in enumerate(tickers):
        ticker_data = price[price['Ticker'] == ticker]['Close']
        
        axes[idx].plot(ticker_data.index, ticker_data.values, linewidth=1.5, color=f'C{idx}')
        axes[idx].set_title(f'{ticker} Close Price Over Time', fontsize=14, fontweight='bold', pad=10)
        axes[idx].set_xlabel('Date', fontsize=12)
        axes[idx].set_ylabel('Price (USD)', fontsize=12)
        axes[idx].grid(True, alpha=0.3)
        
        # Format x-axis dates
        axes[idx].tick_params(axis='x', rotation=45)
        
        # Add some stats as text
        min_price = ticker_data.min()
        max_price = ticker_data.max()
        current_price = ticker_data.iloc[-1]
        axes[idx].text(0.02, 0.98, f'Min: ${min_price:,.2f} | Max: ${max_price:,.2f} | Current: ${current_price:,.2f}',
                      transform=axes[idx].transAxes, fontsize=10, verticalalignment='top',
                      bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available to plot.")

In [ ]:
# Extract metrics from results
# ============================================================================
# DISPLAY MONTE CARLO RESULTS FOR EACH CRYPTOCURRENCY
# ============================================================================

if len(results) > 0:
    # Check if we have results grouped by ticker
    if 'crypto_results' in globals() and len(crypto_results) > 0:
        # Display results for each cryptocurrency separately
        print("\n" + "="*70)
        print(" " * 10 + "MONTE CARLO RESULTS BY CRYPTOCURRENCY")
        print("="*70)
        
        for ticker, ticker_results in crypto_results.items():
            print(f"\n{'='*70}")
            print(f"🪙 {ticker} - MONTE CARLO SIMULATION RESULTS")
            print(f"{'='*70}")
            
            # Extract metrics for this ticker
            sharpe_ratios = [r['sharpe'] for r in ticker_results]
            sortino_ratios = [r['sortino'] for r in ticker_results]
            expected_values = [r['expected_value'] for r in ticker_results]
            returns = [r['return_pct'] for r in ticker_results]
            max_dds = [r['max_dd_pct'] for r in ticker_results]
            pass_rates = [r['passed'] for r in ticker_results]
            win_rates = [r.get('win_rate', 0) for r in ticker_results]
            
            # Calculate statistics
            avg_sharpe = np.mean(sharpe_ratios)
            std_sharpe = np.std(sharpe_ratios)
            median_sharpe = np.median(sharpe_ratios)
            avg_sortino = np.mean(sortino_ratios)
            std_sortino = np.std(sortino_ratios)
            avg_expected_value = np.mean(expected_values)
            std_expected_value = np.std(expected_values)
            avg_win_rate = np.mean(win_rates)
            pass_rate = np.mean(pass_rates) * 100
            
            # Display results
            print(f"\n{'📊 PASS RATE':<30} {pass_rate:>6.1f}% ({sum(pass_rates)}/{len(ticker_results)} simulations passed)")
            
            sharpe_status = "✅ GOOD" if avg_sharpe > 1.0 else "⚠️ MODERATE" if avg_sharpe > 0.5 else "❌ POOR"
            print(f"\n{'🎯 SHARPE RATIO':<30} {sharpe_status}")
            print(f"{'─'*70}")
            print(f"{'   Average:':<25} {avg_sharpe:>8.3f}  {'(>1.0 is good, >2.0 is excellent)'}")
            print(f"{'   Median:':<25} {median_sharpe:>8.3f}")
            print(f"{'   Std Dev:':<25} {std_sharpe:>8.3f}")
            print(f"{'   Range:':<25} [{np.min(sharpe_ratios):>6.3f}, {np.max(sharpe_ratios):>6.3f}]")
            
            ev_status = "✅ PROFITABLE" if avg_expected_value > 0 else "❌ LOSING"
            print(f"\n{'💰 EXPECTED VALUE (Avg PnL per Trade)':<30} {ev_status}")
            print(f"{'─'*70}")
            print(f"{'   Average:':<25} ${avg_expected_value:>7.2f}  {'(Positive = profitable strategy)'}")
            print(f"{'   Std Dev:':<25} ${std_expected_value:>7.2f}")
            print(f"{'   Range:':<25} [${np.min(expected_values):>6.2f}, ${np.max(expected_values):>6.2f}]")
            
            print(f"\n{'📈 SORTINO RATIO':<30}")
            print(f"{'─'*70}")
            print(f"{'   Average:':<25} {avg_sortino:>8.3f}  {'(Downside risk-adjusted return)'}")
            print(f"{'   Std Dev:':<25} {std_sortino:>8.3f}")
            
            win_rate_status = "✅ GOOD" if avg_win_rate > 50 else "⚠️ MODERATE" if avg_win_rate > 40 else "❌ POOR"
            print(f"\n{'🎲 WIN RATE':<30} {win_rate_status}")
            print(f"{'─'*70}")
            print(f"{'   Average:':<25} {avg_win_rate:>8.2f}%  {'(Percentage of winning trades)'}")
            print(f"{'   Std Dev:':<25} {np.std(win_rates):>8.2f}%")
            print(f"{'   Range:':<25} [{np.min(win_rates):>6.2f}%, {np.max(win_rates):>6.2f}%]")
            print(f"{'   Avg Winning Trades:':<25} {np.mean([r.get('winning_trades', 0) for r in ticker_results]):>8.0f}")
            print(f"{'   Avg Losing Trades:':<25} {np.mean([r.get('losing_trades', 0) for r in ticker_results]):>8.0f}")
            
            print(f"\n{'📉 OTHER METRICS':<30}")
            print(f"{'─'*70}")
            print(f"{'   Avg Return:':<25} {np.mean(returns):>8.2f}%")
            print(f"{'   Avg Max DD:':<25} {np.mean(max_dds):>8.2f}%")
            print(f"{'   Avg Trades:':<25} {np.mean([r['num_trades'] for r in ticker_results]):>8.0f}")
            
            # Diagnostic: Why simulations are failing
            if 'failed_reason' in ticker_results[0]:
                failed_reasons = [r.get('failed_reason', 'unknown') for r in ticker_results]
                target_not_met = sum(1 for r in failed_reasons if r == 'target_not_met')
                dd_exceeded = sum(1 for r in failed_reasons if r == 'dd_exceeded')
                passed_count = sum(1 for r in failed_reasons if r == 'passed')
                
                print(f"\n{'🔍 FAILURE ANALYSIS':<30}")
                print(f"{'─'*70}")
                print(f"{'   Passed:':<25} {passed_count:>8} ({passed_count/len(ticker_results)*100:.1f}%)")
                print(f"{'   Failed - Target Not Met:':<25} {target_not_met:>8} ({target_not_met/len(ticker_results)*100:.1f}%)")
                print(f"{'   Failed - DD Exceeded:':<25} {dd_exceeded:>8} ({dd_exceeded/len(ticker_results)*100:.1f}%)")
                
                # Show average values for failed simulations
                if target_not_met > 0:
                    target_failures = [r for r in ticker_results if r.get('failed_reason') == 'target_not_met']
                    avg_pnl = np.mean([r['total_pnl'] for r in target_failures])
                    print(f"{'   Avg PnL (target failures):':<25} ${avg_pnl:>7.2f} (need ${backtester.target_gain})")
                
                if dd_exceeded > 0:
                    dd_failures = [r for r in ticker_results if r.get('failed_reason') == 'dd_exceeded']
                    avg_dd = np.mean([r['max_dd_pct'] for r in dd_failures])
                    print(f"{'   Avg Max DD (DD failures):':<25} {avg_dd:>7.2f}% (limit: {backtester.max_dd_limit*100:.1f}%)")
            
            # Summary table for this ticker
            summary_data = {
                'Metric': ['Sharpe Ratio', 'Sortino Ratio', 'Expected Value ($)', 'Win Rate (%)', 'Return (%)', 'Max DD (%)', 'Pass Rate (%)'],
                'Mean': [f"{avg_sharpe:.3f}", f"{avg_sortino:.3f}", f"${avg_expected_value:.2f}", 
                         f"{avg_win_rate:.2f}", f"{np.mean(returns):.2f}", f"{np.mean(max_dds):.2f}", f"{pass_rate:.1f}"],
                'Std Dev': [f"{std_sharpe:.3f}", f"{std_sortino:.3f}", f"${std_expected_value:.2f}",
                            f"{np.std(win_rates):.2f}", f"{np.std(returns):.2f}", f"{np.std(max_dds):.2f}", "N/A"],
                'Min': [f"{np.min(sharpe_ratios):.3f}", f"{np.min(sortino_ratios):.3f}", 
                        f"${np.min(expected_values):.2f}", f"{np.min(win_rates):.2f}", f"{np.min(returns):.2f}", f"{np.min(max_dds):.2f}", "N/A"],
                'Max': [f"{np.max(sharpe_ratios):.3f}", f"{np.max(sortino_ratios):.3f}",
                        f"${np.max(expected_values):.2f}", f"{np.max(win_rates):.2f}", f"{np.max(returns):.2f}", f"{np.max(max_dds):.2f}", "N/A"]
            }
            summary_df = pd.DataFrame(summary_data)
            print(f"\n📊 {ticker} Summary Statistics Table:")
            print(summary_df.to_string(index=False))
        
        print(f"\n{'='*70}")
        print("✅ ALL CRYPTOCURRENCY RESULTS DISPLAYED")
        print(f"{'='*70}\n")
    
    # Also display combined results
    print(f"\n{'='*70}")
    print(" " * 15 + "COMBINED MONTE CARLO SIMULATION RESULTS")
    print("="*70)
    
if len(results) > 0:
    sharpe_ratios = [r['sharpe'] for r in results]
    sortino_ratios = [r['sortino'] for r in results]
    expected_values = [r['expected_value'] for r in results]
    returns = [r['return_pct'] for r in results]
    max_dds = [r['max_dd_pct'] for r in results]
    pass_rates = [r['passed'] for r in results]
    win_rates = [r.get('win_rate', 0) for r in results]  # Extract win rates
    
    # Calculate statistics
    avg_sharpe = np.mean(sharpe_ratios)
    avg_win_rate = np.mean(win_rates)
    std_sharpe = np.std(sharpe_ratios)
    median_sharpe = np.median(sharpe_ratios)
    
    avg_sortino = np.mean(sortino_ratios)
    std_sortino = np.std(sortino_ratios)
    
    avg_expected_value = np.mean(expected_values)
    std_expected_value = np.std(expected_values)
    
    pass_rate = np.mean(pass_rates) * 100
    
    # Display results with prominent formatting
    print("=" * 70)
    print(" " * 15 + "MONTE CARLO SIMULATION RESULTS")
    print("=" * 70)
    
    print(f"\n{'='*70}")
    print(f"{'📊 PASS RATE':<30} {pass_rate:>6.1f}% ({sum(pass_rates)}/{len(results)} simulations passed)")
    print(f"{'='*70}\n")
    
    # Sharpe Ratio - Prominent Display
    sharpe_status = "✅ GOOD" if avg_sharpe > 1.0 else "⚠️ MODERATE" if avg_sharpe > 0.5 else "❌ POOR"
    print(f"{'🎯 SHARPE RATIO':<30} {sharpe_status}")
    print(f"{'─'*70}")
    print(f"{'   Average:':<25} {avg_sharpe:>8.3f}  {'(>1.0 is good, >2.0 is excellent)'}")
    print(f"{'   Median:':<25} {median_sharpe:>8.3f}")
    print(f"{'   Std Dev:':<25} {std_sharpe:>8.3f}")
    print(f"{'   Range:':<25} [{np.min(sharpe_ratios):>6.3f}, {np.max(sharpe_ratios):>6.3f}]")
    
    # Expected Value - Prominent Display
    ev_status = "✅ PROFITABLE" if avg_expected_value > 0 else "❌ LOSING"
    print(f"\n{'💰 EXPECTED VALUE (Avg PnL per Trade)':<30} {ev_status}")
    print(f"{'─'*70}")
    print(f"{'   Average:':<25} ${avg_expected_value:>7.2f}  {'(Positive = profitable strategy)'}")
    print(f"{'   Std Dev:':<25} ${std_expected_value:>7.2f}")
    print(f"{'   Range:':<25} [${np.min(expected_values):>6.2f}, ${np.max(expected_values):>6.2f}]")
    
    # Sortino Ratio
    print(f"\n{'📈 SORTINO RATIO':<30}")
    print(f"{'─'*70}")
    print(f"{'   Average:':<25} {avg_sortino:>8.3f}  {'(Downside risk-adjusted return)'}")
    print(f"{'   Std Dev:':<25} {std_sortino:>8.3f}")
    
    # Win Rate - Prominent Display
    win_rate_status = "✅ GOOD" if avg_win_rate > 50 else "⚠️ MODERATE" if avg_win_rate > 40 else "❌ POOR"
    print(f"\n{'🎲 WIN RATE':<30} {win_rate_status}")
    print(f"{'─'*70}")
    print(f"{'   Average:':<25} {avg_win_rate:>8.2f}%  {'(Percentage of winning trades)'}")
    print(f"{'   Std Dev:':<25} {np.std(win_rates):>8.2f}%")
    print(f"{'   Range:':<25} [{np.min(win_rates):>6.2f}%, {np.max(win_rates):>6.2f}%]")
    print(f"{'   Avg Winning Trades:':<25} {np.mean([r.get('winning_trades', 0) for r in results]):>8.0f}")
    print(f"{'   Avg Losing Trades:':<25} {np.mean([r.get('losing_trades', 0) for r in results]):>8.0f}")
    
    # Other Metrics
    print(f"\n{'📉 OTHER METRICS':<30}")
    print(f"{'─'*70}")
    print(f"{'   Avg Return:':<25} {np.mean(returns):>8.2f}%")
    print(f"{'   Avg Max DD:':<25} {np.mean(max_dds):>8.2f}%")
    print(f"{'   Avg Trades:':<25} {np.mean([r['num_trades'] for r in results]):>8.0f}")
    
    print(f"\n{'='*70}")
    
    # Create summary DataFrame for easy viewing
    summary_data = {
        'Metric': ['Sharpe Ratio', 'Sortino Ratio', 'Expected Value ($)', 'Win Rate (%)', 'Return (%)', 'Max DD (%)', 'Pass Rate (%)'],
        'Mean': [f"{avg_sharpe:.3f}", f"{avg_sortino:.3f}", f"${avg_expected_value:.2f}", 
                 f"{avg_win_rate:.2f}", f"{np.mean(returns):.2f}", f"{np.mean(max_dds):.2f}", f"{pass_rate:.1f}"],
        'Std Dev': [f"{std_sharpe:.3f}", f"{std_sortino:.3f}", f"${std_expected_value:.2f}",
                    f"{np.std(win_rates):.2f}", f"{np.std(returns):.2f}", f"{np.std(max_dds):.2f}", "N/A"],
        'Min': [f"{np.min(sharpe_ratios):.3f}", f"{np.min(sortino_ratios):.3f}", 
                f"${np.min(expected_values):.2f}", f"{np.min(win_rates):.2f}", f"{np.min(returns):.2f}", f"{np.min(max_dds):.2f}", "N/A"],
        'Max': [f"{np.max(sharpe_ratios):.3f}", f"{np.max(sortino_ratios):.3f}",
                f"${np.max(expected_values):.2f}", f"{np.max(win_rates):.2f}", f"{np.max(returns):.2f}", f"{np.max(max_dds):.2f}", "N/A"]
    }
    summary_df = pd.DataFrame(summary_data)
    
    print("\n📊 Summary Statistics Table:")
    print(summary_df.to_string(index=False))
    
else:
    print("No results available. Run Monte Carlo simulation first.")


## 6. Comprehensive Visualizations

Visual analysis of Monte Carlo simulation results with all available data points.


## 7. Visualizations

Visualize the Monte Carlo results with distributions and key metrics.


In [ ]:
# ============================================================================
# RVOL (RELATIVE VOLUME) STATISTICS
# ============================================================================
# Display volume analysis with RVOL classifications

if 'daily_bias_strategy' in globals() and daily_bias_strategy is not None:
    # Check if we have volume analysis data in context
    # Note: This requires the strategy to have been run with context tracking
    
    print("="*70)
    print(" " * 20 + "RVOL (RELATIVE VOLUME) ANALYSIS")
    print("="*70)
    print()
    print("Volume Classification Thresholds:")
    print("  📈 Ultra-High Volume (Climax): RVOL ≥ 3.0 (300%+ of average)")
    print("  📊 High Volume:           1.5 ≤ RVOL < 3.0 (150-300% of average)")
    print("  ➖ Normal/Average Volume: 0.75 ≤ RVOL < 1.5 (75-150% of average)")
    print("  📉 Low Volume:           0.4 ≤ RVOL < 0.75 (40-75% of average)")
    print("  🔻 Ultra-Low Volume:      RVOL < 0.4 (<40% of average)")
    print()
    print("Calculation:")
    print("  Average Volume = SMA(Volume) over last 20 bars (4H timeframe)")
    print("  RVOL = Current Bar Volume / Average Volume")
    print()
    print("="*70)
    print()
    print("ℹ️  RVOL is calculated automatically during strategy execution.")
    print("   Volume classification is used in 4H candle bias determination.")
    print("   Check strategy context['volume_analysis'] for detailed RVOL data.")
    print()
else:
    print("⚠️  Daily Bias Strategy not loaded. RVOL analysis requires strategy execution.")



## 8. Summary & Interpretation

### 🎯 Key Metrics Explained:

#### **Sharpe Ratio** (Risk-Adjusted Returns)
- **Formula**: (Average Return - Risk-Free Rate) / Standard Deviation of Returns
- **Interpretation**: 
  - **> 2.0**: Excellent strategy
  - **1.0 - 2.0**: Good strategy
  - **0.5 - 1.0**: Moderate strategy
  - **< 0.5**: Poor strategy
- **What it tells you**: How much excess return you get per unit of risk

#### **Expected Value** (Average PnL per Trade)
- **Formula**: Average of all trade PnLs
- **Interpretation**:
  - **Positive**: Strategy is profitable on average
  - **Negative**: Strategy loses money on average
- **What it tells you**: Whether your strategy makes money over many trades

#### **Sortino Ratio** (Downside Risk-Adjusted Returns)
- **Similar to Sharpe but only considers downside volatility**
- **Better for strategies with asymmetric return distributions**
- **> 1.0**: Good downside protection

### 📊 Strategy Assessment Checklist:

Based on the Monte Carlo results, evaluate:

1. **✅ Profitability**: 
   - Expected Value > $0 = Profitable strategy
   - Higher Expected Value = More profitable per trade

2. **✅ Risk-Adjusted Performance**:
   - Sharpe Ratio > 1.0 = Good risk-adjusted returns
   - Higher Sharpe = Better risk-adjusted performance

3. **✅ Consistency**:
   - Low standard deviation in Sharpe/Expected Value = More consistent
   - High standard deviation = Unpredictable results

4. **✅ Risk Management**:
   - Lower Max DD = Better risk control
   - Pass Rate > 60% = Reliable strategy

5. **✅ Reliability**:
   - Higher Pass Rate = More likely to succeed in live trading
   - Consistent positive Expected Value = Sustainable strategy


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from datetime import datetime, timedelta
import pytz
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")

# Load crypto data from .Last.txt files (semicolon-separated format)
# Format: Date Time;Open;High;Low;Close;Volume

data_folders = ["BTCUSD DATA", "ETHUSD DATA", "SOLUSD DATA"]
dfs = []

for folder in data_folders:
    if not os.path.exists(folder):
        print(f"Warning: Folder '{folder}' not found, skipping...")
        continue
    
    # Find all .Last.txt files in the folder
    txt_files = glob.glob(os.path.join(folder, "*.Last.txt"))
    
    for f in txt_files:
        try:
            # Read semicolon-separated file with no header
            df = pd.read_csv(f, sep=';', header=None, 
                           names=['DateTime', 'Open', 'High', 'Low', 'Close', 'Volume'])
            
            # Parse datetime from format: YYYYMMDD HHMMSS
            df['DateTime'] = pd.to_datetime(df['DateTime'], format='%Y%m%d %H%M%S')
            
            # Set datetime as index and sort
            df = df.set_index('DateTime').sort_index()
            
            # Add ticker name from folder
            ticker = folder.replace(' DATA', '').replace('USD', '')
            df['Ticker'] = ticker
            
            dfs.append(df)
            print(f"Loaded {len(df)} rows from {os.path.basename(f)}")
            
        except Exception as e:
            print(f"Error loading {f}: {e}")

# Concatenate all DataFrames if any were loaded
if len(dfs) > 0:
    price = pd.concat(dfs, ignore_index=False)
    print(f"\nTotal rows loaded: {len(price)}")
    print(f"Date range: {price.index.min()} to {price.index.max()}")
    print(f"\nFirst few rows:")
    print(price.head())
else:
    print("No data files found! Please check the data folders.")
    price = pd.DataFrame()  # Create empty DataFrame to avoid errors



In [ ]:
# ============================================================================
# CISD MULTI-TIMEFRAME STRATEGY IMPLEMENTATION
# ============================================================================

NYC_TZ = pytz.timezone('America/New_York')
CENTRAL_TZ = pytz.timezone('America/Chicago')  # Central Timezone (CST/CDT)

class CISDStrategy:
    """
    CISD Multi-Timeframe Strategy
    
    - 4H Chart: Session-profile model for bias (6am-10am NYC session)
    - 5m Chart: CISD signals within active 4H candle
    - Entry: CISD signals aligned with 4H bias
    """
    
    def __init__(self, entry_window_start=6, entry_window_end=10, 
                 htf_timeframe='4H', ltf_timeframe='5m'):
        """
        Parameters:
        -----------
        entry_window_start : int
            Start hour for entry window (NYC time, 24h format)
        entry_window_end : int
            End hour for entry window (NYC time, 24h format)
        htf_timeframe : str
            Higher timeframe (e.g., '4H')
        ltf_timeframe : str
            Lower timeframe (e.g., '5m')
        """
        self.entry_window_start = entry_window_start
        self.entry_window_end = entry_window_end
        self.htf_timeframe = htf_timeframe
        self.ltf_timeframe = ltf_timeframe
        
        # State tracking
        self.current_4h_candle = None
        self.htf_bias = None  # 'bullish', 'bearish', or None
        self.bullish_cisd_signal = False
        self.bearish_cisd_signal = False
        self.recent_bearish_leg_open = None
        self.recent_bullish_leg_open = None
        
    def detect_cisd(self, current_bar, previous_bars, lookback=20):
        """
        Detect CISD (Cumulative Imbalance Supply/Demand) signals.
        
        CISD Logic:
        - Bullish CISD: Price closes above the open of the most recent bearish leg
        - Bearish CISD: Price closes below the open of the most recent bullish leg
        
        Parameters:
        -----------
        current_bar : pd.Series
            Current bar with Open, High, Low, Close
        previous_bars : pd.DataFrame
            Previous bars for lookback analysis
        lookback : int
            Number of bars to look back for leg detection
            
        Returns:
        --------
        tuple: (bullish_cisd, bearish_cisd)
        """
        if len(previous_bars) < 2:
            return False, False
        
        bullish_cisd = False
        bearish_cisd = False
        
        # Find most recent bearish leg (series of lower closes)
        bearish_leg_found = False
        bearish_leg_open = None
        
        for i in range(len(previous_bars) - 1, max(0, len(previous_bars) - lookback) - 1, -1):
            if i < 1:
                break
            bar = previous_bars.iloc[i]
            prev_bar = previous_bars.iloc[i-1]
            
            # Check if this is part of a bearish leg (lower closes)
            if bar['Close'] < prev_bar['Close']:
                if not bearish_leg_found:
                    bearish_leg_open = bar['Open']
                    bearish_leg_found = True
            else:
                if bearish_leg_found:
                    break
        
        # Find most recent bullish leg (series of higher closes)
        bullish_leg_found = False
        bullish_leg_open = None
        
        for i in range(len(previous_bars) - 1, max(0, len(previous_bars) - lookback) - 1, -1):
            if i < 1:
                break
            bar = previous_bars.iloc[i]
            prev_bar = previous_bars.iloc[i-1]
            
            # Check if this is part of a bullish leg (higher closes)
            if bar['Close'] > prev_bar['Close']:
                if not bullish_leg_found:
                    bullish_leg_open = bar['Open']
                    bullish_leg_found = True
            else:
                if bullish_leg_found:
                    break
        
        # Check for Bullish CISD: Price closes above the open of most recent bearish leg
        if bearish_leg_open is not None:
            if current_bar['Close'] > bearish_leg_open:
                bullish_cisd = True
                self.recent_bearish_leg_open = bearish_leg_open
        
        # Check for Bearish CISD: Price closes below the open of most recent bullish leg
        if bullish_leg_open is not None:
            if current_bar['Close'] < bullish_leg_open:
                bearish_cisd = True
                self.recent_bullish_leg_open = bullish_leg_open
        
        return bullish_cisd, bearish_cisd
    
    def determine_4h_bias(self, htf_data, current_time):
        """
        Determine 4H bias using session-profile model.
        
        Focus on 6am-10am NYC session 4H candle.
        Bias is determined by the direction and strength of that candle.
        
        Parameters:
        -----------
        htf_data : pd.DataFrame
            4H timeframe data
        current_time : pd.Timestamp
            Current time to identify active 4H candle
            
        Returns:
        --------
        str: 'bullish', 'bearish', or None
        """
        if len(htf_data) == 0:
            return None
        
        # Convert to NYC time
        if current_time.tzinfo is None:
            current_time = current_time.tz_localize('UTC')
        nyc_time = current_time.tz_convert(NYC_TZ)
        
        # Find 4H candles that overlap with 6am-10am NYC session
        # 4H candles: 0:00, 4:00, 8:00, 12:00, 16:00, 20:00 NYC time
        # We want the candle that contains 6am-10am: that's the 4:00-8:00 or 8:00-12:00 candle
        
        target_hour = nyc_time.hour
        if target_hour >= 6 and target_hour < 10:
            # We're in the 6am-10am window
            # Find the 4H candle that started at 4am or 8am
            candle_start_hour = 4 if target_hour < 8 else 8
        else:
            # Find the most recent 4H candle
            candle_start_hour = (target_hour // 4) * 4
        
        # Find matching 4H candle
        htf_data_nyc = htf_data.copy()
        if htf_data_nyc.index.tzinfo is None:
            htf_data_nyc.index = htf_data_nyc.index.tz_localize('UTC')
        htf_data_nyc.index = htf_data_nyc.index.tz_convert(NYC_TZ)
        
        # Filter to target hour
        matching_candles = htf_data_nyc[htf_data_nyc.index.hour == candle_start_hour]
        
        if len(matching_candles) == 0:
            return None
        
        # Get the most recent matching candle
        recent_candle = matching_candles.iloc[-1]
        
        # Determine bias based on candle direction and strength
        candle_range = recent_candle['High'] - recent_candle['Low']
        candle_body = abs(recent_candle['Close'] - recent_candle['Open'])
        body_ratio = candle_body / candle_range if candle_range > 0 else 0
        
        # Bullish if close > open and strong body
        if recent_candle['Close'] > recent_candle['Open'] and body_ratio > 0.5:
            return 'bullish'
        # Bearish if close < open and strong body
        elif recent_candle['Close'] < recent_candle['Open'] and body_ratio > 0.5:
            return 'bearish'
        # Weak bias if body is small
        else:
            # Use overall trend
            if recent_candle['Close'] > recent_candle['Open']:
                return 'bullish'
            else:
                return 'bearish'
    
    def is_in_entry_window(self, current_time):
        """
        Check if current time is within the entry window.
        
        Parameters:
        -----------
        current_time : pd.Timestamp
            Current timestamp
            
        Returns:
        --------
        bool: True if in entry window
        """
        if current_time.tzinfo is None:
            current_time = current_time.tz_localize('UTC')
        nyc_time = current_time.tz_convert(NYC_TZ)
        hour = nyc_time.hour
        
        return self.entry_window_start <= hour < self.entry_window_end
    
    def check_entry_signal(self, current_bar, previous_bars, current_time, htf_data):
        """
        Check for entry signal based on CISD and 4H bias.
        
        Entry Rules:
        - Bullish: 4H bias bullish AND Bullish CISD formed within entry window
        - Bearish: 4H bias bearish AND Bearish CISD formed within entry window
        
        Parameters:
        -----------
        current_bar : pd.Series
            Current 5m bar
        previous_bars : pd.DataFrame
            Previous 5m bars
        current_time : pd.Timestamp
            Current time
        htf_data : pd.DataFrame
            4H timeframe data
            
        Returns:
        --------
        tuple: (signal, confidence)
            signal: 'BUY', 'SELL', or 'HOLD'
            confidence: float between 0 and 1
        """
        # Check if in entry window
        if not self.is_in_entry_window(current_time):
            return 'HOLD', 0.5
        
        # Determine 4H bias
        htf_bias = self.determine_4h_bias(htf_data, current_time)
        
        if htf_bias is None:
            return 'HOLD', 0.5
        
        # Detect CISD signals
        bullish_cisd, bearish_cisd = self.detect_cisd(current_bar, previous_bars)
        
        # Entry logic
        if htf_bias == 'bullish' and bullish_cisd:
            # Bullish 4H bias + Bullish CISD = Long entry
            return 'BUY', 0.85
        elif htf_bias == 'bearish' and bearish_cisd:
            # Bearish 4H bias + Bearish CISD = Short entry
            return 'SELL', 0.85
        else:
            # No alignment or no CISD signal
            return 'HOLD', 0.5

# Initialize CISD strategy
cisd_strategy = CISDStrategy(
    entry_window_start=6,  # 6am NYC
    entry_window_end=10,   # 10am NYC
    htf_timeframe='4H',
    ltf_timeframe='5m'
)

print("✅ CISD Multi-Timeframe Strategy initialized!")
print(f"   Entry Window: {cisd_strategy.entry_window_start}:00 - {cisd_strategy.entry_window_end}:00 NYC time")
print(f"   Higher Timeframe: {cisd_strategy.htf_timeframe}")
print(f"   Lower Timeframe: {cisd_strategy.ltf_timeframe}")


In [ ]:
# ============================================================================
# INTERACTIVE PARAMETER TESTING - MODIFY VALUES BELOW
# ============================================================================

# Copy and modify these parameters to test different scenarios
TEST_PARAMS = {
    # Account settings
    'account_size': 5000,          # Starting account size ($)
    'entry_size': 4800,             # Position size per trade ($)
    'target_gain': 500,             # Target profit to reach ($)
    
    # Risk limits (as decimals: 0.02 = 2%)
    'daily_dd_limit': 0.02,         # Maximum daily drawdown (2%)
    'max_dd_limit': 0.03,           # Maximum total drawdown (3%)
    
    # Monte Carlo settings
    'num_simulations': 100,         # Number of Monte Carlo runs (lower for faster testing)
    'sample_size': 50,              # Number of trades per simulation
}

# Display current test parameters
print("=" * 70)
print("TESTING WITH FOLLOWING PARAMETERS:")
print("=" * 70)
for key, value in TEST_PARAMS.items():
    if isinstance(value, float):
        print(f"  {key:25s}: {value:.4f} ({value*100:.2f}%)")
    else:
        print(f"  {key:25s}: {value}")
print("=" * 70)

# Initialize backtester with test parameters
test_backtester = CryptoBacktester(
    account_size=TEST_PARAMS['account_size'],
    daily_dd_limit=TEST_PARAMS['daily_dd_limit'],
    max_dd_limit=TEST_PARAMS['max_dd_limit'],
    entry_size=TEST_PARAMS['entry_size'],
    target_gain=TEST_PARAMS['target_gain']
)

# Quick test run (if trades_df exists)
if 'trades_df' in globals() and len(trades_df) > 0:
    print(f"\nRunning quick test with {TEST_PARAMS['num_simulations']} simulations...")
    test_results = []
    
    for i in range(TEST_PARAMS['num_simulations']):
        # Randomly sample trades
        sample_trades = trades_df.sample(n=min(TEST_PARAMS['sample_size'], len(trades_df)), replace=True)
        sample_trades = sample_trades.sort_values('entry_time').reset_index(drop=True)
        
        # Run backtest (using strategy if defined)
        strategy_func = my_strategy if 'my_strategy' in globals() else None
        result = test_backtester.run_backtest(sample_trades, strategy_func=strategy_func)
        test_results.append(result)
        
        if (i + 1) % 25 == 0:
            print(f"  Completed {i + 1}/{TEST_PARAMS['num_simulations']} simulations...")
    
    # Calculate metrics
    test_df = pd.DataFrame(test_results)
# Add calculated columns for compatibility (backtester returns different key names)
    # This fixes the KeyError by mapping backtester column names to expected names
    if 'equity_curve' in test_df.columns:
        test_df['final_equity'] = test_df['equity_curve'].apply(lambda x: x[-1] if len(x) > 0 else TEST_PARAMS['account_size'])
    if 'sharpe' in test_df.columns:
        test_df['sharpe_ratio'] = test_df['sharpe']
    if 'sortino' in test_df.columns:
        test_df['sortino_ratio'] = test_df['sortino']
    if 'max_dd_pct' in test_df.columns:
        test_df['max_drawdown'] = test_df['max_dd_pct'] / 100  # Convert percentage to decimal
    
    print("\n" + "=" * 70)
    print("QUICK TEST RESULTS:")
    print("=" * 70)
    print(f"  Expected Value:      ${test_df['final_equity'].mean() - TEST_PARAMS['account_size']:.2f}")
    print(f"  Pass Rate:           {test_df['passed'].mean() * 100:.2f}%")
    print(f"  Avg Final Equity:   ${test_df['final_equity'].mean():.2f}")
    print(f"  Sharpe Ratio:        {test_df['sharpe_ratio'].mean():.4f}")
    print(f"  Max Drawdown:        {test_df['max_drawdown'].mean() * 100:.2f}%")
    print("=" * 70)
    
    # Store results for visualization
    test_results_df = test_df
else:
    print("\n⚠ Warning: trades_df not found. Please run the trade preparation cell first.")
    test_results_df = pd.DataFrame()


In [ ]:
# ============================================================================
# VISUALIZE PARAMETER IMPACT
# ============================================================================

if 'test_results_df' in globals() and len(test_results_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Final Equity Distribution
    axes[0, 0].hist(test_results_df['final_equity'], bins=30, edgecolor='black', alpha=0.7, color='skyblue')
    axes[0, 0].axvline(TEST_PARAMS['account_size'], color='red', linestyle='--', linewidth=2, label='Starting Equity')
    axes[0, 0].axvline(test_results_df['final_equity'].mean(), color='green', linestyle='--', linewidth=2, label='Mean Final Equity')
    axes[0, 0].set_xlabel('Final Equity ($)', fontsize=11)
    axes[0, 0].set_ylabel('Frequency', fontsize=11)
    axes[0, 0].set_title('Final Equity Distribution', fontsize=12, fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Pass/Fail Rate
    pass_rate = test_results_df['passed'].mean()
    axes[0, 1].bar(['Passed', 'Failed'], 
                   [pass_rate, 1 - pass_rate], 
                   color=['green', 'red'], alpha=0.7, edgecolor='black')
    axes[0, 1].set_ylabel('Proportion', fontsize=11)
    axes[0, 1].set_title(f'Pass Rate: {pass_rate*100:.1f}%', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylim([0, 1])
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # 3. Sharpe Ratio Distribution
    axes[1, 0].hist(test_results_df['sharpe_ratio'], bins=30, edgecolor='black', alpha=0.7, color='orange')
    axes[1, 0].axvline(test_results_df['sharpe_ratio'].mean(), color='red', linestyle='--', linewidth=2, label='Mean Sharpe')
    axes[1, 0].axvline(1.0, color='green', linestyle='--', linewidth=1, label='Good Threshold (1.0)')
    axes[1, 0].set_xlabel('Sharpe Ratio', fontsize=11)
    axes[1, 0].set_ylabel('Frequency', fontsize=11)
    axes[1, 0].set_title('Sharpe Ratio Distribution', fontsize=12, fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Drawdown Analysis
    axes[1, 1].hist(test_results_df['max_drawdown'] * 100, bins=30, edgecolor='black', alpha=0.7, color='coral')
    axes[1, 1].axvline(TEST_PARAMS['max_dd_limit'] * 100, color='red', linestyle='--', linewidth=2, label='Max DD Limit')
    axes[1, 1].set_xlabel('Max Drawdown (%)', fontsize=11)
    axes[1, 1].set_ylabel('Frequency', fontsize=11)
    axes[1, 1].set_title('Maximum Drawdown Distribution', fontsize=12, fontweight='bold')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.suptitle(f'Strategy Performance with Current Parameters\n'
                 f'Account Size: ${TEST_PARAMS["account_size"]:,} | '
                 f'Entry Size: ${TEST_PARAMS["entry_size"]:,} | '
                 f'Target Gain: ${TEST_PARAMS["target_gain"]:,}', 
                 fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
    
    # Summary statistics table
    print("\n" + "=" * 70)
    print("DETAILED STATISTICS:")
    print("=" * 70)
    summary_stats = {
        'Metric': ['Expected Value', 'Pass Rate', 'Avg Final Equity', 'Std Final Equity', 
                   'Avg Sharpe Ratio', 'Avg Max Drawdown', 'Avg Sortino Ratio'],
        'Value': [
            f"${test_results_df['final_equity'].mean() - TEST_PARAMS['account_size']:.2f}",
            f"{test_results_df['passed'].mean() * 100:.2f}%",
            f"${test_results_df['final_equity'].mean():.2f}",
            f"${test_results_df['final_equity'].std():.2f}",
            f"{test_results_df['sharpe_ratio'].mean():.4f}",
            f"{test_results_df['max_drawdown'].mean() * 100:.2f}%",
            f"{test_results_df['sortino_ratio'].mean():.4f}"
        ]
    }
    summary_df = pd.DataFrame(summary_stats)
    print(summary_df.to_string(index=False))
    print("=" * 70)
else:
    print("No test results available. Please run the parameter testing cell above first.")


In [ ]:
# ============================================================================
# MONTE CARLO RESULTS VISUALIZATION
# ============================================================================

if len(results) > 0:
    # Extract metrics
    sharpe_ratios = [r['sharpe'] for r in results]
    sortino_ratios = [r['sortino'] for r in results]
    expected_values = [r['expected_value'] for r in results]
    returns = [r['return_pct'] for r in results]
    max_dds = [r['max_dd_pct'] for r in results]
    pass_rates = [r['passed'] for r in results]
    
    # Calculate statistics
    avg_sharpe = np.mean(sharpe_ratios)
    median_sharpe = np.median(sharpe_ratios)
    std_sharpe = np.std(sharpe_ratios)
    
    avg_expected_value = np.mean(expected_values)
    std_expected_value = np.std(expected_values)
    
    avg_sortino = np.mean(sortino_ratios)
    pass_rate = np.mean(pass_rates) * 100
    
    # Create comprehensive visualization
    fig = plt.figure(figsize=(18, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.3)
    
    # 1. Sharpe Ratio Distribution (Top Left - Large)
    ax1 = fig.add_subplot(gs[0, 0])
    n, bins, patches = ax1.hist(sharpe_ratios, bins=40, alpha=0.7, color='green', 
                               edgecolor='black', linewidth=1.2)
    ax1.axvline(avg_sharpe, color='red', linestyle='--', linewidth=3, 
               label=f'Mean: {avg_sharpe:.3f}')
    ax1.axvline(median_sharpe, color='blue', linestyle='--', linewidth=2, 
               label=f'Median: {median_sharpe:.3f}')
    ax1.axvline(1.0, color='orange', linestyle='--', linewidth=2, 
               label='Good: >1.0')
    ax1.axvline(2.0, color='gold', linestyle=':', linewidth=2, 
               label='Excellent: >2.0')
    ax1.set_xlabel('Sharpe Ratio', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Frequency', fontsize=13)
    ax1.set_title('🎯 Sharpe Ratio Distribution\n(Risk-Adjusted Returns)', 
                 fontsize=14, fontweight='bold', pad=10)
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    # Color bars
    for i, (patch, bin_val) in enumerate(zip(patches, bins[:-1])):
        if bin_val >= 2.0:
            patch.set_facecolor('darkgreen')
        elif bin_val >= 1.0:
            patch.set_facecolor('green')
        elif bin_val >= 0.5:
            patch.set_facecolor('lightgreen')
        else:
            patch.set_facecolor('lightcoral')
    
    # 2. Expected Value Distribution (Top Middle)
    ax2 = fig.add_subplot(gs[0, 1])
    n2, bins2, patches2 = ax2.hist(expected_values, bins=40, alpha=0.7, color='blue', 
                                   edgecolor='black', linewidth=1.2)
    ax2.axvline(avg_expected_value, color='red', linestyle='--', linewidth=3, 
               label=f'Mean: ${avg_expected_value:.2f}')
    ax2.axvline(0, color='black', linestyle='-', linewidth=2, alpha=0.7, 
               label='Break-even')
    ax2.set_xlabel('Expected Value ($)', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Frequency', fontsize=13)
    ax2.set_title('💰 Expected Value Distribution\n(Avg PnL per Trade)', 
                 fontsize=14, fontweight='bold', pad=10)
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)
    # Color bars
    for i, (patch, bin_val) in enumerate(zip(patches2, bins2[:-1])):
        if bin_val >= 0:
            patch.set_facecolor('steelblue')
        else:
            patch.set_facecolor('lightcoral')
    
    # 3. Pass Rate Visualization (Top Right)
    ax3 = fig.add_subplot(gs[0, 2])
    passed = sum(pass_rates)
    failed = len(results) - passed
    colors = ['green' if pass_rate > 50 else 'red', 'lightcoral']
    ax3.bar(['Passed', 'Failed'], [passed, failed], color=colors, alpha=0.7, 
           edgecolor='black', linewidth=2)
    ax3.set_ylabel('Number of Simulations', fontsize=12, fontweight='bold')
    ax3.set_title(f'📊 Pass Rate: {pass_rate:.1f}%\n({passed}/{len(results)} passed)', 
                 fontsize=14, fontweight='bold', pad=10)
    ax3.grid(True, alpha=0.3, axis='y')
    # Add value labels
    for i, (label, val) in enumerate(zip(['Passed', 'Failed'], [passed, failed])):
        ax3.text(i, val + len(results)*0.01, str(val), ha='center', 
                va='bottom', fontweight='bold', fontsize=12)
    
    # 4. Return Distribution (Middle Left)
    ax4 = fig.add_subplot(gs[1, 0])
    ax4.hist(returns, bins=40, alpha=0.7, color='purple', edgecolor='black')
    ax4.axvline(np.mean(returns), color='red', linestyle='--', linewidth=2, 
               label=f'Mean: {np.mean(returns):.2f}%')
    ax4.axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)
    ax4.set_xlabel('Return (%)', fontsize=12)
    ax4.set_ylabel('Frequency', fontsize=12)
    ax4.set_title('📈 Return Distribution', fontsize=13, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Max Drawdown Distribution (Middle Center)
    ax5 = fig.add_subplot(gs[1, 1])
    ax5.hist(max_dds, bins=40, alpha=0.7, color='red', edgecolor='black')
    ax5.axvline(np.mean(max_dds), color='blue', linestyle='--', linewidth=2, 
               label=f'Mean: {np.mean(max_dds):.2f}%')
    ax5.axvline(BACKTEST_PARAMS['max_dd_limit']*100, color='orange', 
               linestyle='--', linewidth=2, label=f'Limit: {BACKTEST_PARAMS["max_dd_limit"]*100:.1f}%')
    ax5.set_xlabel('Max Drawdown (%)', fontsize=12)
    ax5.set_ylabel('Frequency', fontsize=12)
    ax5.set_title('📉 Max Drawdown Distribution', fontsize=13, fontweight='bold')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # 6. Sortino Ratio Distribution (Middle Right)
    ax6 = fig.add_subplot(gs[1, 2])
    ax6.hist(sortino_ratios, bins=40, alpha=0.7, color='teal', edgecolor='black')
    ax6.axvline(avg_sortino, color='red', linestyle='--', linewidth=2, 
               label=f'Mean: {avg_sortino:.3f}')
    ax6.axvline(1.0, color='orange', linestyle='--', linewidth=1, label='Good: >1.0')
    ax6.set_xlabel('Sortino Ratio', fontsize=12)
    ax6.set_ylabel('Frequency', fontsize=12)
    ax6.set_title('📊 Sortino Ratio Distribution', fontsize=13, fontweight='bold')
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    
    # 7. Key Metrics Summary (Bottom - Span all columns)
    ax7 = fig.add_subplot(gs[2, :])
    metrics = ['Sharpe\nRatio', 'Expected\nValue ($)', 'Sortino\nRatio', 
              'Return\n(%)', 'Max DD\n(%)', 'Pass\nRate (%)']
    values = [avg_sharpe, avg_expected_value/10, avg_sortino, 
             np.mean(returns), np.mean(max_dds), pass_rate/10]
    actual_values = [avg_sharpe, avg_expected_value, avg_sortino, 
                    np.mean(returns), np.mean(max_dds), pass_rate]
    colors = ['green' if v > 0 else 'red' for v in [avg_sharpe, avg_expected_value, 
                                                    avg_sortino, np.mean(returns), 
                                                    -np.mean(max_dds), pass_rate-50]]
    bars = ax7.bar(metrics, values, color=colors, alpha=0.7, 
                  edgecolor='black', linewidth=1.5)
    ax7.axhline(0, color='black', linestyle='-', linewidth=1)
    ax7.set_ylabel('Normalized Value', fontsize=12)
    ax7.set_title('📊 Key Metrics Summary Comparison', fontsize=14, fontweight='bold')
    ax7.grid(True, alpha=0.3, axis='y')
    # Add value labels
    for i, (bar, val, orig_val) in enumerate(zip(bars, values, actual_values)):
        height = bar.get_height()
        if i == 1:  # Expected Value
            label = f'${orig_val:.2f}'
        elif i == 5:  # Pass Rate
            label = f'{orig_val:.1f}%'
        elif i == 3:  # Return
            label = f'{orig_val:.2f}%'
        elif i == 4:  # Max DD
            label = f'{orig_val:.2f}%'
        else:
            label = f'{orig_val:.3f}'
        ax7.text(bar.get_x() + bar.get_width()/2., height,
                label, ha='center', va='bottom' if height > 0 else 'top', 
                fontweight='bold', fontsize=11)
    
    # Add parameter info as text
    param_text = (f"Strategy Params: High={STRATEGY_PARAMS['high_threshold']:.2f}, "
                 f"Mid={STRATEGY_PARAMS['mid_threshold']:.2f} | "
                 f"Backtest: Account=${BACKTEST_PARAMS['account_size']:,}, "
                 f"DD Limit={BACKTEST_PARAMS['max_dd_limit']*100:.1f}%")
    plt.suptitle('Monte Carlo Simulation Results - Parameter Impact Analysis', 
                 fontsize=16, fontweight='bold', y=0.995)
    fig.text(0.5, 0.02, param_text, ha='center', fontsize=10, 
            style='italic', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.show()
    
    # Print summary statistics
    print("\n" + "=" * 70)
    print("SUMMARY STATISTICS")
    print("=" * 70)
    print(f"Sharpe Ratio:      Mean={avg_sharpe:.3f}, Median={median_sharpe:.3f}, Std={std_sharpe:.3f}")
    print(f"Expected Value:    Mean=${avg_expected_value:.2f}, Std=${std_expected_value:.2f}")
    print(f"Sortino Ratio:    Mean={avg_sortino:.3f}")
    print(f"Return:            Mean={np.mean(returns):.2f}%, Std={np.std(returns):.2f}%")
    print(f"Max Drawdown:     Mean={np.mean(max_dds):.2f}%, Std={np.std(max_dds):.2f}%")
    print(f"Pass Rate:        {pass_rate:.1f}% ({passed}/{len(results)} simulations)")
    print("=" * 70)
    
else:
    print("❌ No results to visualize. Run Monte Carlo simulation first.")


In [ ]:
# ============================================================================
# COMPREHENSIVE MONTE CARLO VISUALIZATIONS
# ============================================================================

if len(results) > 0:
    # Extract all metrics
    sharpe_ratios = [r['sharpe'] for r in results]
    sortino_ratios = [r['sortino'] for r in results]
    expected_values = [r['expected_value'] for r in results]
    returns = [r['return_pct'] for r in results]
    max_dds = [r['max_dd_pct'] for r in results]
    pass_rates = [r['passed'] for r in results]
    num_trades_list = [r['num_trades'] for r in results]
    
    # Create figure with subplots
    fig = plt.figure(figsize=(20, 16))
    gs = fig.add_gridspec(4, 3, hspace=0.3, wspace=0.3)
    
    # 1. Sharpe Ratio Distribution
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.hist(sharpe_ratios, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    ax1.axvline(np.mean(sharpe_ratios), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(sharpe_ratios):.3f}')
    ax1.axvline(np.median(sharpe_ratios), color='green', linestyle='--', linewidth=2, label=f'Median: {np.median(sharpe_ratios):.3f}')
    ax1.axvline(1.0, color='orange', linestyle=':', linewidth=2, label='Good Threshold (1.0)')
    ax1.set_xlabel('Sharpe Ratio', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax1.set_title('Sharpe Ratio Distribution', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Sortino Ratio Distribution
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.hist(sortino_ratios, bins=50, color='coral', alpha=0.7, edgecolor='black')
    ax2.axvline(np.mean(sortino_ratios), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(sortino_ratios):.3f}')
    ax2.axvline(np.median(sortino_ratios), color='green', linestyle='--', linewidth=2, label=f'Median: {np.median(sortino_ratios):.3f}')
    ax2.set_xlabel('Sortino Ratio', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax2.set_title('Sortino Ratio Distribution', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Expected Value Distribution
    ax3 = fig.add_subplot(gs[0, 2])
    ax3.hist(expected_values, bins=50, color='mediumseagreen', alpha=0.7, edgecolor='black')
    ax3.axvline(np.mean(expected_values), color='red', linestyle='--', linewidth=2, label=f'Mean: ${np.mean(expected_values):.2f}')
    ax3.axvline(0, color='black', linestyle='-', linewidth=1, label='Break-even')
    ax3.set_xlabel('Expected Value ($)', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax3.set_title('Expected Value Distribution', fontsize=14, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Return Distribution
    ax4 = fig.add_subplot(gs[1, 0])
    ax4.hist(returns, bins=50, color='gold', alpha=0.7, edgecolor='black')
    ax4.axvline(np.mean(returns), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(returns):.2f}%')
    ax4.axvline(0, color='black', linestyle='-', linewidth=1, label='Break-even')
    ax4.set_xlabel('Return (%)', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax4.set_title('Return Distribution', fontsize=14, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Max Drawdown Distribution
    ax5 = fig.add_subplot(gs[1, 1])
    ax5.hist(max_dds, bins=50, color='tomato', alpha=0.7, edgecolor='black')
    ax5.axvline(np.mean(max_dds), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(max_dds):.2f}%')
    ax5.set_xlabel('Max Drawdown (%)', fontsize=12, fontweight='bold')
    ax5.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax5.set_title('Max Drawdown Distribution', fontsize=14, fontweight='bold')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # 6. Pass Rate Visualization
    ax6 = fig.add_subplot(gs[1, 2])
    pass_count = sum(pass_rates)
    fail_count = len(pass_rates) - pass_count
    colors = ['#2ecc71', '#e74c3c']
    ax6.pie([pass_count, fail_count], labels=[f'Passed ({pass_count})', f'Failed ({fail_count})'], 
            autopct='%1.1f%%', colors=colors, startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
    ax6.set_title(f'Pass Rate: {np.mean(pass_rates)*100:.1f}%', fontsize=14, fontweight='bold')
    
    # 7. Sharpe vs Sortino Scatter
    ax7 = fig.add_subplot(gs[2, 0])
    scatter = ax7.scatter(sharpe_ratios, sortino_ratios, c=expected_values, cmap='RdYlGn', 
                         alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
    ax7.axvline(1.0, color='orange', linestyle=':', linewidth=1, alpha=0.5)
    ax7.axhline(1.0, color='orange', linestyle=':', linewidth=1, alpha=0.5)
    ax7.set_xlabel('Sharpe Ratio', fontsize=12, fontweight='bold')
    ax7.set_ylabel('Sortino Ratio', fontsize=12, fontweight='bold')
    ax7.set_title('Sharpe vs Sortino Ratio', fontsize=14, fontweight='bold')
    plt.colorbar(scatter, ax=ax7, label='Expected Value ($)')
    ax7.grid(True, alpha=0.3)
    
    # 8. Expected Value vs Return
    ax8 = fig.add_subplot(gs[2, 1])
    scatter2 = ax8.scatter(expected_values, returns, c=sharpe_ratios, cmap='viridis', 
                          alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
    ax8.axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)
    ax8.axhline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)
    ax8.set_xlabel('Expected Value ($)', fontsize=12, fontweight='bold')
    ax8.set_ylabel('Return (%)', fontsize=12, fontweight='bold')
    ax8.set_title('Expected Value vs Return', fontsize=14, fontweight='bold')
    plt.colorbar(scatter2, ax=ax8, label='Sharpe Ratio')
    ax8.grid(True, alpha=0.3)
    
    # 9. Number of Trades Distribution
    ax9 = fig.add_subplot(gs[2, 2])
    ax9.hist(num_trades_list, bins=30, color='purple', alpha=0.7, edgecolor='black')
    ax9.axvline(np.mean(num_trades_list), color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {np.mean(num_trades_list):.0f}')
    ax9.set_xlabel('Number of Trades', fontsize=12, fontweight='bold')
    ax9.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax9.set_title('Number of Trades Distribution', fontsize=14, fontweight='bold')
    ax9.legend()
    ax9.grid(True, alpha=0.3)
    
    # 10. Cumulative Distribution Functions
    ax10 = fig.add_subplot(gs[3, :])
    sorted_sharpe = np.sort(sharpe_ratios)
    p = np.arange(1, len(sorted_sharpe) + 1) / len(sorted_sharpe)
    ax10.plot(sorted_sharpe, p, linewidth=2, label='Sharpe Ratio CDF', color='steelblue')
    
    sorted_ev = np.sort(expected_values)
    p_ev = np.arange(1, len(sorted_ev) + 1) / len(sorted_ev)
    ax10_twin = ax10.twinx()
    ax10_twin.plot(sorted_ev, p_ev, linewidth=2, label='Expected Value CDF', color='mediumseagreen')
    
    ax10.axvline(1.0, color='orange', linestyle=':', linewidth=1, alpha=0.5, label='Sharpe = 1.0')
    ax10.set_xlabel('Value', fontsize=12, fontweight='bold')
    ax10.set_ylabel('Cumulative Probability (Sharpe)', fontsize=12, fontweight='bold', color='steelblue')
    ax10_twin.set_ylabel('Cumulative Probability (EV)', fontsize=12, fontweight='bold', color='mediumseagreen')
    ax10.set_title('Cumulative Distribution Functions', fontsize=14, fontweight='bold')
    ax10.legend(loc='upper left')
    ax10_twin.legend(loc='upper right')
    ax10.grid(True, alpha=0.3)
    
    plt.suptitle(f'Monte Carlo Simulation Results - {NUM_SIMULATIONS} Simulations, {SAMPLE_SIZE:,} Trades per Simulation\n'
                 f'Total Data Points Analyzed: {NUM_SIMULATIONS * SAMPLE_SIZE:,}', 
                 fontsize=16, fontweight='bold', y=0.995)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*80)
    print("VISUALIZATION SUMMARY")
    print("="*80)
    print(f"Total Simulations: {NUM_SIMULATIONS:,}")
    print(f"Trades per Simulation: {SAMPLE_SIZE:,}")
    print(f"Total Data Points Analyzed: {NUM_SIMULATIONS * SAMPLE_SIZE:,}")
    print(f"Sharpe Ratio > 1.0: {sum(1 for s in sharpe_ratios if s > 1.0)} ({sum(1 for s in sharpe_ratios if s > 1.0)/len(sharpe_ratios)*100:.1f}%)")
    print(f"Expected Value > 0: {sum(1 for ev in expected_values if ev > 0)} ({sum(1 for ev in expected_values if ev > 0)/len(expected_values)*100:.1f}%)")
    print(f"Pass Rate: {np.mean(pass_rates)*100:.1f}%")
    print("="*80)
    
else:
    print("No results available. Run Monte Carlo simulation first.")


In [ ]:
# ============================================================================
# MONTE CARLO SIMULATION - FAN-STYLE VISUALIZATION WITH FILTER BUTTON
# ============================================================================
# This creates a fan-style plot showing all equity curves from Monte Carlo simulations
# Similar to the image provided, showing multiple price paths diverging over time
# Includes an interactive button to filter to show only top 5 and bottom 5 paths

try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("⚠️ ipywidgets not available. Install with: pip install ipywidgets")
    print("   Button functionality will be disabled.")

if len(results) > 0:
    # Extract all equity curves from results (all 5000 simulations will be plotted)
    equity_curves = [r['equity_curve'] for r in results if 'equity_curve' in r and len(r['equity_curve']) > 0]
    
    print(f"📊 Plotting {len(equity_curves):,} equity curves from Monte Carlo simulations...")
    
    if len(equity_curves) > 0:
        # Find the maximum length to normalize all curves
        max_length = max(len(ec) for ec in equity_curves)
        
        # Create figure with white background (like the reference image)
        fig, ax = plt.subplots(figsize=(14, 8))
        fig.patch.set_facecolor('white')
        ax.set_facecolor('white')
        
        # Get account size for initial value
        account_size = BACKTEST_PARAMS.get('account_size', 5000) if 'BACKTEST_PARAMS' in globals() else 5000
        
        # Plot each equity curve as a separate line
        # Use a colormap to get different colors for each line
        # For 5000 simulations, use a larger colormap and adjust alpha for better visibility
        num_curves = len(equity_curves)
        colors = plt.cm.viridis(np.linspace(0, 1, num_curves))  # Use viridis for better color distribution
        
        # Adjust line width and alpha based on number of curves
        line_width = 0.5 if num_curves > 1000 else 0.8
        line_alpha = 0.4 if num_curves > 1000 else 0.7
        
        print(f"   Plotting {num_curves:,} lines (this may take a moment)...")
        
        for i, equity_curve in enumerate(equity_curves):
            # Normalize to same length (pad with last value if needed)
            if len(equity_curve) < max_length:
                equity_curve = equity_curve + [equity_curve[-1]] * (max_length - len(equity_curve))
            
            # Create time steps (x-axis)
            time_steps = np.arange(len(equity_curve))
            
            # Plot the line with a distinct color
            ax.plot(time_steps, equity_curve, linewidth=line_width, alpha=line_alpha, color=colors[i])
            
            # Progress indicator for large numbers
            if num_curves > 1000 and (i + 1) % 1000 == 0:
                print(f"   Plotted {i + 1:,}/{num_curves:,} curves...")
        
        # Set labels and title (matching the reference image style)
        ax.set_xlabel('Time Steps', fontsize=12, fontweight='bold')
        ax.set_ylabel('Equity in USD', fontsize=12, fontweight='bold')
        ax.set_title(f'Monte Carlo Simulation - {len(equity_curves)} Equity Paths', 
                     fontsize=14, fontweight='bold', pad=15)
        
        # Add grid for better readability
        ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
        
        # Set y-axis limits to show full range (with some padding)
        if len(equity_curves) > 0:
            all_values = [val for ec in equity_curves for val in ec]
            y_min = min(all_values)
            y_max = max(all_values)
            y_range = y_max - y_min
            ax.set_ylim(max(0, y_min - y_range * 0.1), y_max + y_range * 0.1)
        
        # Format y-axis to show values in thousands if needed
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
        
        # Add a reference line at starting account size
        ax.axhline(y=account_size, color='black', linestyle='-', linewidth=1.5, 
                  alpha=0.5, label=f'Starting Equity: ${account_size:,.0f}')
        
        # Add legend
        ax.legend(loc='upper left', fontsize=10)
        
        plt.tight_layout()
        plt.show()
        
        # Store equity curves and results for filtering
        stored_equity_curves = equity_curves.copy()
        stored_results = results.copy()
        
        # Create interactive button for filtering
        if WIDGETS_AVAILABLE:
            # Create output area for the filtered plot
            output = widgets.Output()
            
            def plot_filtered_paths(button):
                """Filter and plot only top 5 and bottom 5 paths."""
                with output:
                    clear_output(wait=True)
                    
                    # Calculate final equity for each curve
                    final_equities = [(i, ec[-1]) for i, ec in enumerate(stored_equity_curves)]
                    
                    # Sort by final equity
                    final_equities.sort(key=lambda x: x[1])
                    
                    # Get indices of top 5 and bottom 5
                    bottom_5_indices = [idx for idx, _ in final_equities[:5]]
                    top_5_indices = [idx for idx, _ in final_equities[-5:]]
                    
                    # Combine indices
                    filtered_indices = set(bottom_5_indices + top_5_indices)
                    
                    # Filter equity curves
                    filtered_curves = [stored_equity_curves[i] for i in filtered_indices]
                    filtered_results = [stored_results[i] for i in filtered_indices]
                    
                    # Create new plot
                    fig, ax = plt.subplots(figsize=(14, 8))
                    fig.patch.set_facecolor('white')
                    ax.set_facecolor('white')
                    
                    # Find max length
                    max_length = max(len(ec) for ec in filtered_curves)
                    
                    # Plot bottom 5 (worst) in red shades
                    bottom_5_curves = [stored_equity_curves[i] for i in bottom_5_indices]
                    for i, equity_curve in enumerate(bottom_5_curves):
                        if len(equity_curve) < max_length:
                            equity_curve = equity_curve + [equity_curve[-1]] * (max_length - len(equity_curve))
                        time_steps = np.arange(len(equity_curve))
                        ax.plot(time_steps, equity_curve, linewidth=2, alpha=0.8, 
                               color=plt.cm.Reds(0.3 + i * 0.15), 
                               label=f'Worst #{i+1}: ${equity_curve[-1]:,.0f}')
                    
                    # Plot top 5 (best) in green shades
                    top_5_curves = [stored_equity_curves[i] for i in top_5_indices]
                    for i, equity_curve in enumerate(top_5_curves):
                        if len(equity_curve) < max_length:
                            equity_curve = equity_curve + [equity_curve[-1]] * (max_length - len(equity_curve))
                        time_steps = np.arange(len(equity_curve))
                        ax.plot(time_steps, equity_curve, linewidth=2, alpha=0.8, 
                               color=plt.cm.Greens(0.3 + i * 0.15),
                               label=f'Best #{i+1}: ${equity_curve[-1]:,.0f}')
                    
                    # Set labels and title
                    ax.set_xlabel('Time Steps', fontsize=12, fontweight='bold')
                    ax.set_ylabel('Equity in USD', fontsize=12, fontweight='bold')
                    ax.set_title('Monte Carlo Simulation - Top 5 Best & Bottom 5 Worst Paths', 
                               fontsize=14, fontweight='bold', pad=15)
                    
                    # Add grid
                    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
                    
                    # Set y-axis limits
                    all_values = [val for ec in filtered_curves for val in ec]
                    y_min = min(all_values)
                    y_max = max(all_values)
                    y_range = y_max - y_min
                    ax.set_ylim(max(0, y_min - y_range * 0.1), y_max + y_range * 0.1)
                    
                    # Format y-axis
                    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
                    
                    # Add reference line
                    ax.axhline(y=account_size, color='black', linestyle='-', linewidth=1.5, 
                              alpha=0.5, label=f'Starting Equity: ${account_size:,.0f}')
                    
                    # Add legend
                    ax.legend(loc='upper left', fontsize=9, ncol=2)
                    
                    plt.tight_layout()
                    plt.show()
                    
                    # Print summary
                    print(f"\n{'='*70}")
                    print(f"FILTERED MONTE CARLO PATHS (Top 5 & Bottom 5)")
                    print(f"{'='*70}")
                    print(f"\n📉 Bottom 5 Worst Paths:")
                    for i, idx in enumerate(bottom_5_indices):
                        final_equity = stored_equity_curves[idx][-1]
                        return_pct = ((final_equity - account_size) / account_size) * 100
                        print(f"   #{i+1}: Final Equity: ${final_equity:,.2f} ({return_pct:+.2f}%)")
                    
                    print(f"\n📈 Top 5 Best Paths:")
                    for i, idx in enumerate(reversed(top_5_indices)):
                        final_equity = stored_equity_curves[idx][-1]
                        return_pct = ((final_equity - account_size) / account_size) * 100
                        print(f"   #{i+1}: Final Equity: ${final_equity:,.2f} ({return_pct:+.2f}%)")
                    print(f"{'='*70}\n")
            
            # Create button
            filter_button = widgets.Button(
                description='Filter: Show Top 5 & Bottom 5',
                button_style='info',
                tooltip='Click to filter visualization to show only the 5 best and 5 worst paths',
                icon='filter',
                layout=widgets.Layout(width='250px', height='40px')
            )
            
            filter_button.on_click(plot_filtered_paths)
            
            # Display button
            print(f"\n{'='*70}")
            print("INTERACTIVE FILTER BUTTON")
            print(f"{'='*70}")
            display(filter_button)
            display(output)
            print(f"{'='*70}\n")
        
        # Print summary
        print(f"\n{'='*70}")
        print(f"MONTE CARLO FAN PLOT SUMMARY")
        print(f"{'='*70}")
        print(f"Total Simulations Plotted: {len(equity_curves):,}")
        print(f"Starting Equity: ${account_size:,.2f}")
        if len(equity_curves) > 0:
            final_values = [ec[-1] for ec in equity_curves]
            print(f"Final Equity Range: ${min(final_values):,.2f} - ${max(final_values):,.2f}")
            print(f"Average Final Equity: ${np.mean(final_values):,.2f}")
            print(f"Median Final Equity: ${np.median(final_values):,.2f}")
        print(f"{'='*70}\n")
    else:
        print("No equity curves found in results.")
else:
    print("No results available. Run Monte Carlo simulation first.")



In [ ]:
# ============================================================================
# VALIDATION & CROSS-CHECK OF MONTE CARLO RESULTS
# ============================================================================
# This cell validates the accuracy of all calculated metrics and results

if len(results) > 0:
    print("="*70)
    print(" " * 15 + "RESULTS VALIDATION & CROSS-CHECK")
    print("="*70)
    print()
    
    # Get account size
    account_size = BACKTEST_PARAMS.get('account_size', 5000) if 'BACKTEST_PARAMS' in globals() else 5000
    target_gain = BACKTEST_PARAMS.get('target_gain', 500) if 'BACKTEST_PARAMS' in globals() else 500
    max_dd_limit = BACKTEST_PARAMS.get('max_dd_limit', 0.03) if 'BACKTEST_PARAMS' in globals() else 0.03
    
    validation_errors = []
    validation_warnings = []
    
    # 1. VALIDATE EQUITY CURVES
    print("1️⃣  VALIDATING EQUITY CURVES")
    print("-" * 70)
    equity_issues = 0
    for i, result in enumerate(results):
        if 'equity_curve' not in result or len(result['equity_curve']) == 0:
            validation_errors.append(f"Result {i}: Missing or empty equity curve")
            equity_issues += 1
            continue
        
        equity_curve = result['equity_curve']
        
        # Check: Equity curve should start at account_size
        if equity_curve[0] != account_size:
            validation_warnings.append(f"Result {i}: Equity curve doesn't start at account_size ({equity_curve[0]} vs {account_size})")
        
        # Check: Equity should never go negative
        if min(equity_curve) < 0:
            validation_errors.append(f"Result {i}: Equity went negative (min: {min(equity_curve):.2f})")
        
        # Check: Final equity should match reported final_equity
        if abs(equity_curve[-1] - result.get('final_equity', 0)) > 0.01:
            validation_errors.append(f"Result {i}: Final equity mismatch (curve: {equity_curve[-1]:.2f}, reported: {result.get('final_equity', 0):.2f})")
    
    print(f"   ✓ Checked {len(results)} equity curves")
    if equity_issues == 0:
        print("   ✅ All equity curves are valid")
    else:
        print(f"   ⚠️  Found {equity_issues} equity curve issues")
    print()
    
    # 2. VALIDATE PNL CALCULATIONS
    print("2️⃣  VALIDATING PNL CALCULATIONS")
    print("-" * 70)
    pnl_issues = 0
    for i, result in enumerate(results):
        final_equity = result.get('final_equity', account_size)
        total_pnl = result.get('total_pnl', 0)
        expected_pnl = final_equity - account_size
        
        # Check: total_pnl should equal final_equity - account_size
        if abs(total_pnl - expected_pnl) > 0.01:
            validation_errors.append(f"Result {i}: PnL mismatch (reported: {total_pnl:.2f}, expected: {expected_pnl:.2f})")
            pnl_issues += 1
        
        # Check: return_pct should match
        return_pct = result.get('return_pct', 0)
        expected_return = (total_pnl / account_size) * 100
        if abs(return_pct - expected_return) > 0.01:
            validation_errors.append(f"Result {i}: Return % mismatch (reported: {return_pct:.2f}%, expected: {expected_return:.2f}%)")
            pnl_issues += 1
    
    print(f"   ✓ Checked {len(results)} PnL calculations")
    if pnl_issues == 0:
        print("   ✅ All PnL calculations are correct")
    else:
        print(f"   ❌ Found {pnl_issues} PnL calculation errors")
    print()
    
    # 3. VALIDATE DRAWDOWN CALCULATIONS
    print("3️⃣  VALIDATING DRAWDOWN CALCULATIONS")
    print("-" * 70)
    dd_issues = 0
    for i, result in enumerate(results):
        if 'equity_curve' not in result or len(result['equity_curve']) < 2:
            continue
        
        equity_curve = np.array(result['equity_curve'])
        reported_max_dd = result.get('max_dd_pct', 0)
        
        # Recalculate max drawdown
        rolling_max = np.maximum.accumulate(equity_curve)
        drawdowns = (rolling_max - equity_curve) / rolling_max
        calculated_max_dd = drawdowns.max() * 100 if len(drawdowns) > 0 else 0
        
        # Check: Max DD should match
        if abs(reported_max_dd - calculated_max_dd) > 0.01:
            validation_errors.append(f"Result {i}: Max DD mismatch (reported: {reported_max_dd:.2f}%, calculated: {calculated_max_dd:.2f}%)")
            dd_issues += 1
        
        # Check: Drawdown should never exceed 100%
        if calculated_max_dd > 100:
            validation_errors.append(f"Result {i}: Max DD exceeds 100% ({calculated_max_dd:.2f}%)")
            dd_issues += 1
    
    print(f"   ✓ Checked {len(results)} drawdown calculations")
    if dd_issues == 0:
        print("   ✅ All drawdown calculations are correct")
    else:
        print(f"   ❌ Found {dd_issues} drawdown calculation errors")
    print()
    
    # 4. VALIDATE WIN RATE CALCULATIONS
    print("4️⃣  VALIDATING WIN RATE CALCULATIONS")
    print("-" * 70)
    wr_issues = 0
    for i, result in enumerate(results):
        if 'trade_log' not in result or len(result['trade_log']) == 0:
            continue
        
        trade_log = result['trade_log']
        reported_win_rate = result.get('win_rate', 0)
        reported_winning = result.get('winning_trades', 0)
        reported_losing = result.get('losing_trades', 0)
        
        # Recalculate win rate
        winning_trades = sum(1 for t in trade_log if t.get('pnl', 0) > 0)
        losing_trades = sum(1 for t in trade_log if t.get('pnl', 0) < 0)
        total_trades = len(trade_log)
        calculated_win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0
        
        # Check: Win rate should match
        if abs(reported_win_rate - calculated_win_rate) > 0.01:
            validation_errors.append(f"Result {i}: Win rate mismatch (reported: {reported_win_rate:.2f}%, calculated: {calculated_win_rate:.2f}%)")
            wr_issues += 1
        
        # Check: Winning/losing trade counts
        if reported_winning != winning_trades:
            validation_errors.append(f"Result {i}: Winning trades mismatch (reported: {reported_winning}, calculated: {winning_trades})")
            wr_issues += 1
        
        if reported_losing != losing_trades:
            validation_errors.append(f"Result {i}: Losing trades mismatch (reported: {reported_losing}, calculated: {losing_trades})")
            wr_issues += 1
    
    print(f"   ✓ Checked {len(results)} win rate calculations")
    if wr_issues == 0:
        print("   ✅ All win rate calculations are correct")
    else:
        print(f"   ❌ Found {wr_issues} win rate calculation errors")
    print()
    
    # 5. VALIDATE EXPECTED VALUE CALCULATIONS
    print("5️⃣  VALIDATING EXPECTED VALUE CALCULATIONS")
    print("-" * 70)
    ev_issues = 0
    for i, result in enumerate(results):
        if 'trade_log' not in result or len(result['trade_log']) == 0:
            continue
        
        trade_log = result['trade_log']
        reported_ev = result.get('expected_value', 0)
        
        # Recalculate expected value
        pnls = [t.get('pnl', 0) for t in trade_log]
        calculated_ev = np.mean(pnls) if len(pnls) > 0 else 0
        
        # Check: Expected value should match
        if abs(reported_ev - calculated_ev) > 0.01:
            validation_errors.append(f"Result {i}: Expected value mismatch (reported: ${reported_ev:.2f}, calculated: ${calculated_ev:.2f})")
            ev_issues += 1
    
    print(f"   ✓ Checked {len(results)} expected value calculations")
    if ev_issues == 0:
        print("   ✅ All expected value calculations are correct")
    else:
        print(f"   ❌ Found {ev_issues} expected value calculation errors")
    print()
    
    # 6. VALIDATE SHARPE RATIO CALCULATIONS
    print("6️⃣  VALIDATING SHARPE RATIO CALCULATIONS")
    print("-" * 70)
    sharpe_issues = 0
    for i, result in enumerate(results):
        if 'equity_curve' not in result or len(result['equity_curve']) < 2:
            continue
        
        equity_curve = np.array(result['equity_curve'])
        reported_sharpe = result.get('sharpe', 0)
        
        # Recalculate Sharpe ratio
        returns = np.diff(equity_curve) / equity_curve[:-1]
        returns = returns[~np.isnan(returns)]
        
        if len(returns) > 1 and returns.std() > 0:
            calculated_sharpe = (returns.mean() * np.sqrt(312)) / returns.std()
        else:
            calculated_sharpe = 0
        
        # Check: Sharpe should match (allow small tolerance for floating point)
        if abs(reported_sharpe - calculated_sharpe) > 0.001:
            validation_errors.append(f"Result {i}: Sharpe ratio mismatch (reported: {reported_sharpe:.4f}, calculated: {calculated_sharpe:.4f})")
            sharpe_issues += 1
    
    print(f"   ✓ Checked {len(results)} Sharpe ratio calculations")
    if sharpe_issues == 0:
        print("   ✅ All Sharpe ratio calculations are correct")
    else:
        print(f"   ❌ Found {sharpe_issues} Sharpe ratio calculation errors")
    print()
    
    # 7. VALIDATE PASS/FAIL CRITERIA
    print("7️⃣  VALIDATING PASS/FAIL CRITERIA")
    print("-" * 70)
    pass_issues = 0
    for i, result in enumerate(results):
        total_pnl = result.get('total_pnl', 0)
        max_dd = result.get('max_dd_pct', 0)
        reported_passed = result.get('passed', False)
        
        # Recalculate pass criteria
        calculated_passed = total_pnl >= target_gain and max_dd < max_dd_limit * 100
        
        # Check: Pass status should match
        if reported_passed != calculated_passed:
            validation_errors.append(f"Result {i}: Pass status mismatch (reported: {reported_passed}, calculated: {calculated_passed}, PnL: ${total_pnl:.2f}, DD: {max_dd:.2f}%)")
            pass_issues += 1
    
    print(f"   ✓ Checked {len(results)} pass/fail criteria")
    if pass_issues == 0:
        print("   ✅ All pass/fail criteria are correct")
    else:
        print(f"   ❌ Found {pass_issues} pass/fail criteria errors")
    print()
    
    # 8. STATISTICAL CONSISTENCY CHECKS
    print("8️⃣  STATISTICAL CONSISTENCY CHECKS")
    print("-" * 70)
    
    # Extract all metrics
    final_equities = [r.get('final_equity', account_size) for r in results]
    total_pnls = [r.get('total_pnl', 0) for r in results]
    sharpe_ratios = [r.get('sharpe', 0) for r in results]
    expected_values = [r.get('expected_value', 0) for r in results]
    win_rates = [r.get('win_rate', 0) for r in results]
    max_dds = [r.get('max_dd_pct', 0) for r in results]
    num_trades_list = [r.get('num_trades', 0) for r in results]
    
    # Check: PnL should equal final_equity - account_size for all
    pnl_consistency = all(abs(pnl - (eq - account_size)) < 0.01 for pnl, eq in zip(total_pnls, final_equities))
    print(f"   PnL Consistency: {'✅ PASS' if pnl_consistency else '❌ FAIL'}")
    
    # Check: Win rate should be between 0 and 100
    wr_range = all(0 <= wr <= 100 for wr in win_rates)
    print(f"   Win Rate Range (0-100%): {'✅ PASS' if wr_range else '❌ FAIL'}")
    
    # Check: Max DD should be between 0 and 100
    dd_range = all(0 <= dd <= 100 for dd in max_dds)
    print(f"   Max DD Range (0-100%): {'✅ PASS' if dd_range else '❌ FAIL'}")
    
    # Check: Number of trades should be non-negative
    trades_positive = all(nt >= 0 for nt in num_trades_list)
    print(f"   Number of Trades (>=0): {'✅ PASS' if trades_positive else '❌ FAIL'}")
    
    print()
    
    # 9. SUMMARY STATISTICS VERIFICATION
    print("9️⃣  SUMMARY STATISTICS VERIFICATION")
    print("-" * 70)
    
    # Calculate summary stats
    avg_sharpe = np.mean(sharpe_ratios)
    avg_ev = np.mean(expected_values)
    avg_wr = np.mean(win_rates)
    avg_pnl = np.mean(total_pnls)
    pass_rate = np.mean([r.get('passed', False) for r in results]) * 100
    
    print(f"   Average Sharpe Ratio: {avg_sharpe:.4f}")
    print(f"   Average Expected Value: ${avg_ev:.2f}")
    print(f"   Average Win Rate: {avg_wr:.2f}%")
    print(f"   Average PnL: ${avg_pnl:.2f}")
    print(f"   Pass Rate: {pass_rate:.2f}%")
    print(f"   Total Simulations: {len(results):,}")
    print()
    
    # 10. FINAL VALIDATION SUMMARY
    print("="*70)
    print(" " * 20 + "VALIDATION SUMMARY")
    print("="*70)
    
    if len(validation_errors) == 0 and len(validation_warnings) == 0:
        print("✅ ALL VALIDATIONS PASSED - Results are accurate!")
    else:
        if len(validation_errors) > 0:
            print(f"❌ FOUND {len(validation_errors)} VALIDATION ERRORS:")
            for error in validation_errors[:10]:  # Show first 10 errors
                print(f"   - {error}")
            if len(validation_errors) > 10:
                print(f"   ... and {len(validation_errors) - 10} more errors")
        
        if len(validation_warnings) > 0:
            print(f"\n⚠️  FOUND {len(validation_warnings)} VALIDATION WARNINGS:")
            for warning in validation_warnings[:5]:  # Show first 5 warnings
                print(f"   - {warning}")
            if len(validation_warnings) > 5:
                print(f"   ... and {len(validation_warnings) - 5} more warnings")
    
    print("="*70)
    print()
    
    # 11. MANUAL VERIFICATION SAMPLE
    print("🔍 MANUAL VERIFICATION SAMPLE")
    print("-" * 70)
    print("Selecting 3 random results for detailed manual verification:")
    print()
    
    sample_indices = np.random.choice(len(results), size=min(3, len(results)), replace=False)
    for idx in sample_indices:
        result = results[idx]
        print(f"Result #{idx}:")
        print(f"  Final Equity: ${result.get('final_equity', 0):.2f}")
        print(f"  Total PnL: ${result.get('total_pnl', 0):.2f}")
        print(f"  Return: {result.get('return_pct', 0):.2f}%")
        print(f"  Max DD: {result.get('max_dd_pct', 0):.2f}%")
        print(f"  Sharpe: {result.get('sharpe', 0):.4f}")
        print(f"  Expected Value: ${result.get('expected_value', 0):.2f}")
        print(f"  Win Rate: {result.get('win_rate', 0):.2f}%")
        print(f"  Number of Trades: {result.get('num_trades', 0)}")
        print(f"  Passed: {result.get('passed', False)}")
        
        # Manual calculation check
        if 'equity_curve' in result and len(result['equity_curve']) > 0:
            manual_pnl = result['equity_curve'][-1] - account_size
            manual_return = (manual_pnl / account_size) * 100
            print(f"  Manual Check - PnL: ${manual_pnl:.2f}, Return: {manual_return:.2f}%")
        
        print()
    
else:
    print("❌ No results available for validation. Run Monte Carlo simulation first.")



## 9. Results Validation & Cross-Checking

Use the tools below to verify the validity and accuracy of Monte Carlo simulation results.


In [ ]:
# Create comprehensive visualizations
if len(results) > 0:
    # Create figure with 2x2 layout, emphasizing Sharpe and Expected Value
    fig = plt.figure(figsize=(16, 12))
    gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)
    
    # 1. Sharpe Ratio Distribution (Top Left - Prominent)
    ax1 = fig.add_subplot(gs[0, 0])
    n, bins, patches = ax1.hist(sharpe_ratios, bins=30, alpha=0.7, color='green', edgecolor='black', linewidth=1.2)
    ax1.axvline(avg_sharpe, color='red', linestyle='--', linewidth=3, label=f'Mean: {avg_sharpe:.3f}')
    ax1.axvline(1.0, color='orange', linestyle='--', linewidth=2, label='Good Threshold: >1.0')
    ax1.axvline(2.0, color='gold', linestyle=':', linewidth=2, label='Excellent: >2.0')
    ax1.set_xlabel('Sharpe Ratio', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Frequency', fontsize=13)
    ax1.set_title('🎯 Sharpe Ratio Distribution\n(Risk-Adjusted Returns)', fontsize=15, fontweight='bold', pad=10)
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    # Color bars based on value
    for i, (patch, bin_val) in enumerate(zip(patches, bins[:-1])):
        if bin_val >= 1.0:
            patch.set_facecolor('darkgreen')
        elif bin_val >= 0.5:
            patch.set_facecolor('lightgreen')
        else:
            patch.set_facecolor('lightcoral')
    
    # 2. Expected Value Distribution (Top Right - Prominent)
    ax2 = fig.add_subplot(gs[0, 1])
    n2, bins2, patches2 = ax2.hist(expected_values, bins=30, alpha=0.7, color='blue', edgecolor='black', linewidth=1.2)
    ax2.axvline(avg_expected_value, color='red', linestyle='--', linewidth=3, label=f'Mean: ${avg_expected_value:.2f}')
    ax2.axvline(0, color='black', linestyle='-', linewidth=2, alpha=0.7, label='Break-even')
    ax2.set_xlabel('Expected Value ($)', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Frequency', fontsize=13)
    ax2.set_title('💰 Expected Value Distribution\n(Average PnL per Trade)', fontsize=15, fontweight='bold', pad=10)
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
    # Color bars: green for positive, red for negative
    for i, (patch, bin_val) in enumerate(zip(patches2, bins2[:-1])):
        if bin_val >= 0:
            patch.set_facecolor('steelblue')
        else:
            patch.set_facecolor('lightcoral')
    
    # 3. Sortino Ratio Distribution (Middle Left)
    ax3 = fig.add_subplot(gs[1, 0])
    ax3.hist(sortino_ratios, bins=30, alpha=0.7, color='teal', edgecolor='black')
    ax3.axvline(avg_sortino, color='red', linestyle='--', linewidth=2, label=f'Mean: {avg_sortino:.3f}')
    ax3.axvline(1.0, color='orange', linestyle='--', linewidth=1, label='Good: >1.0')
    ax3.set_xlabel('Sortino Ratio', fontsize=12)
    ax3.set_ylabel('Frequency', fontsize=12)
    ax3.set_title('📈 Sortino Ratio Distribution\n(Downside Risk-Adjusted)', fontsize=13, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Return Distribution (Middle Right)
    ax4 = fig.add_subplot(gs[1, 1])
    ax4.hist(returns, bins=30, alpha=0.7, color='purple', edgecolor='black')
    ax4.axvline(np.mean(returns), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(returns):.2f}%')
    ax4.axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)
    ax4.set_xlabel('Return (%)', fontsize=12)
    ax4.set_ylabel('Frequency', fontsize=12)
    ax4.set_title('📉 Return Distribution', fontsize=13, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Key Metrics Comparison (Bottom - Span both columns)
    ax5 = fig.add_subplot(gs[2, :])
    metrics = ['Sharpe\nRatio', 'Expected\nValue ($)', 'Sortino\nRatio', 'Return\n(%)', 'Max DD\n(%)']
    values = [avg_sharpe, avg_expected_value/10, avg_sortino, np.mean(returns), np.mean(max_dds)]  # Scale EV for visibility
    colors = ['green' if v > 0 else 'red' for v in [avg_sharpe, avg_expected_value, avg_sortino, np.mean(returns), -np.mean(max_dds)]]
    bars = ax5.bar(metrics, values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
    ax5.axhline(0, color='black', linestyle='-', linewidth=1)
    ax5.set_ylabel('Normalized Value', fontsize=12)
    ax5.set_title('📊 Key Metrics Comparison', fontsize=14, fontweight='bold')
    ax5.grid(True, alpha=0.3, axis='y')
    # Add value labels on bars
    for i, (bar, val, orig_val) in enumerate(zip(bars, values, [avg_sharpe, avg_expected_value, avg_sortino, np.mean(returns), np.mean(max_dds)])):
        height = bar.get_height()
        label = f'{orig_val:.2f}' if i != 1 else f'${orig_val:.2f}'
        ax5.text(bar.get_x() + bar.get_width()/2., height,
                label, ha='center', va='bottom' if height > 0 else 'top', fontweight='bold')
    
    plt.suptitle('Monte Carlo Simulation Results - Key Metrics Visualization', 
                 fontsize=16, fontweight='bold', y=0.995)
    plt.show()
    
else:
    print("No results to visualize. Run Monte Carlo simulation first.")


In [ ]:
# Plot Close prices with proper labels
plt.figure(figsize=(12, 6))

# Check if we have multiple tickers
if 'Ticker' in price.columns:
    tickers = price['Ticker'].unique()
    if len(tickers) > 1:
        # Plot each ticker separately with labels
        for ticker in tickers:
            ticker_data = price[price['Ticker'] == ticker]['Close']
            plt.plot(ticker_data.index, ticker_data.values, label=f'{ticker} Close Price', alpha=0.7)
        plt.legend()
        plt.title('Cryptocurrency Close Prices Over Time', fontsize=14, fontweight='bold')
    else:
        # Single ticker
        plt.plot(price['Close'], label=f'{tickers[0]} Close Price')
        plt.legend()
        plt.title(f'{tickers[0]} Close Price Over Time', fontsize=14, fontweight='bold')
else:
    # No ticker column, just plot all data
    plt.plot(price['Close'], label='Close Price')
    plt.legend()
    plt.title('Close Price Over Time', fontsize=14, fontweight='bold')

plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()